In [66]:
#vérifier si l'identifiant COM correspond à une donnée dans codes_geo_2024 sans sensibilité aux majuscules
#afficher le nom de la table + la commune qui est dans les tables des facteurs mais qui manque dans codes_geo_2024
#chercher le nom de la commune dans la table des données de transitions et mettre à jour l'ancien nom avec le nom de 2024
# verifier que l'ancien nom est bien devenu celui de 2024 et qu'il n'y a plus d'erreurs

# head(data) pour chaque table
# commentaires expliquant les colonnes qu'on garde et suppression de celles inutiles : restructuration

# faire une moyenne par facteur (colonne) pour chaque commune, et vérification d'avoir qu'1 colonne par facteur 

#merger en 1 table unique


In [67]:
# codgeo partout 

In [68]:
import os
import glob
import pandas as pd
import csv
import importlib
import utils
importlib.reload(utils)
from utils import detect_delimiter, detect_header


In [69]:
# Importation des facteurs avec granularité "COMMUNE", affichage des premières lignes, et détermination des headers

In [70]:
dossier = os.path.join(os.getcwd(), 'facteurs', 'commune')

if not os.path.exists(dossier):
    print(f"❌ Le dossier '{dossier}' n'existe pas.")
else:
    fichiers = glob.glob(os.path.join(dossier, '*'))

    # Exclure StockEtablissement_utf8 dès le départ
    fichiers = [f for f in fichiers if 'StockEtablissement_utf8' not in os.path.basename(f)]

    total_fichiers = len(fichiers)
    print(f"📁 Nombre de fichiers trouvés (hors StockEtablissement_utf8) : {total_fichiers}\n")

    for idx, fichier in enumerate(fichiers, start=1):
        nom = os.path.basename(fichier)
        nom_sans_extension = os.path.splitext(nom)[0]

        print(f"🔄 Traitement du fichier {idx}/{total_fichiers} : {nom}")  # affichage progression

        try:
            df = None

            # -----------------------------
            # CSV
            # -----------------------------
            if fichier.lower().endswith('.csv'):
                sep = detect_delimiter(fichier)
                header_ligne = detect_header(fichier, sep=sep)
                df = pd.read_csv(
                    fichier,
                    sep=sep,
                    header=header_ligne,
                    engine='python',
                    quotechar='"',
                    quoting=csv.QUOTE_MINIMAL,
                    on_bad_lines='skip',
                    dtype=str
                )
                print(f'✅ {nom} importé avec succès (CSV, sep="{sep}", header={header_ligne})')

            # -----------------------------
            # Excel (xls, xlsx, xlsm)
            # -----------------------------
            elif fichier.lower().endswith(('.xls', '.xlsx', '.xlsm')):
                header_ligne = detect_header(fichier)
                engine = 'openpyxl' if fichier.lower().endswith(('.xlsx', '.xlsm')) else 'xlrd'
                df = pd.read_excel(fichier, header=header_ligne, engine=engine)
                print(f'✅ {nom} importé avec succès (Excel, header={header_ligne})')

            # -----------------------------
            # Format non supporté
            # -----------------------------
            else:
                print(f'❌ Format non supporté : {nom}')

            # -----------------------------
            # Stockage global
            # -----------------------------
            if df is not None:
                globals()[nom_sans_extension] = df
                print(df.head())
                print("--------")

        except Exception as e:
            print(f'❌ Erreur import {nom} : {e}')
            try:
                with open(fichier, 'r', encoding='utf-8', errors='replace') as f:
                    print("📄 Aperçu du contenu :")
                    for _ in range(5):
                        print(f.readline().strip())
            except:
                print("⚠️ Impossible d'afficher le contenu du fichier.")
            print("--------")


📁 Nombre de fichiers trouvés (hors StockEtablissement_utf8) : 20

🔄 Traitement du fichier 1/20 : altitude_communes.csv
✅ altitude_communes.csv importé avec succès (CSV, sep=",", header=0)
  Unnamed: 0 code_insee             nom_standard        nom_sans_pronom  \
0          0      01001  L'Abergement-Clémenciat  Abergement-Clémenciat   
1          1      01002    L'Abergement-de-Varey    Abergement-de-Varey   
2          2      01004        Ambérieu-en-Bugey      Ambérieu-en-Bugey   
3          3      01005      Ambérieux-en-Dombes    Ambérieux-en-Dombes   
4          4      01006                  Ambléon                Ambléon   

                     nom_a                      nom_de  \
0  à Abergement-Clémenciat  de l'Abergement-Clémenciat   
1    à Abergement-de-Varey    de l'Abergement-de-Varey   
2      à Ambérieu-en-Bugey         d'Ambérieu-en-Bugey   
3    à Ambérieux-en-Dombes       d'Ambérieux-en-Dombes   
4                à Ambléon                   d'Ambléon   

           n

In [71]:
# importation speciale pour StockEtablissement_utf8
dossier = os.path.join(os.getcwd(), 'facteurs', 'commune')
chemin_fichier = os.path.join(dossier, 'StockEtablissement_utf8.csv')
StockEtablissement_utf8 = pd.read_csv(chemin_fichier, sep=",", dtype=str)

In [72]:
codes_geo_2024 = "insee_millesime_importation/v_commune_2024.csv"
sep = detect_delimiter(codes_geo_2024)
codes_geo_2024 = pd.read_csv(codes_geo_2024, sep=sep, low_memory=False)
print(f'✅ {codes_geo_2024} importé avec succès. (CSV, sep="{sep}")')
print("\n📊 Aperçu des données :")
print(codes_geo_2024.head())

✅       TYPECOM    COM   REG  DEP  CTCD  ARR  TNCC                    NCC  \
0         COM  01001  84.0   01   01D  012     5  ABERGEMENT CLEMENCIAT   
1         COM  01002  84.0   01   01D  011     5    ABERGEMENT DE VAREY   
2         COM  01004  84.0   01   01D  011     1      AMBERIEU EN BUGEY   
3         COM  01005  84.0   01   01D  012     1    AMBERIEUX EN DOMBES   
4         COM  01006  84.0   01   01D  011     1                AMBLEON   
...       ...    ...   ...  ...   ...  ...   ...                    ...   
37539     COM  97613   6.0  976  976D  NaN     0          M TSANGAMOUJI   
37540     COM  97614   6.0  976  976D  NaN     1               OUANGANI   
37541     COM  97615   6.0  976  976D  NaN     0               PAMANDZI   
37542     COM  97616   6.0  976  976D  NaN     0                   SADA   
37543     COM  97617   6.0  976  976D  NaN     0               TSINGONI   

                      NCCENR                  LIBELLE    CAN  COMPARENT  
0      Abergement-Cléme

In [73]:
#correction des headers
employeurs_salaries[["codgeo", "Libelle"]] = employeurs_salaries["Commune"].str.split(" ", n=1, expand=True)
salaire_secteur_COM_2022.reset_index(drop=True, inplace=True)
salaire_secteur_COM_2022 = salaire_secteur_COM_2022.drop(index=0)
salaire_secteur_COM_2022.reset_index(drop=True, inplace=True)

# Renommer la colonne "Code géographique" en "codgeo"
infrastructures_sportives.rename(columns={"new_code": "codgeo"}, inplace=True)
salaire_secteur_COM_2022.rename(columns={"Code géographique": "codgeo"}, inplace=True)
#Educ_COM_2023.rename(columns={"CODGEO": "codgeo"}, inplace=True)
indicateur_population_COM_2015_23.rename(columns={"CODGEO": "codgeo"}, inplace=True)
StockEtablissement_utf8 = StockEtablissement_utf8.rename(columns={"codeCommuneEtablissement": "codgeo"})
tourisme_capacite.rename(columns={"GEO": "codgeo"}, inplace=True)
tourisme_capacite = tourisme_capacite[tourisme_capacite['GEO_OBJECT'].str.startswith('COM')]
age_insee.rename(columns={"INSEE": "codgeo"}, inplace=True)
altitude_communes.rename(columns={"code_insee": "codgeo"}, inplace=True)
impots_communes.rename(columns={"code INSEE": "codgeo"}, inplace=True)

In [74]:
# Vérification de l'existence des valeurs de 'codgeo' dans 'COM' de 'codes_geo_2024' en ignorant la casse
for fichier in fichiers:
    nom = os.path.basename(fichier)
    nom_sans_extension = os.path.splitext(nom)[0]  # Extrait le nom sans extension

    if nom_sans_extension in globals():  # Vérifie que le DataFrame est bien chargé dans l'environnement
        df = globals()[nom_sans_extension]
        
        # Vérifie si la colonne 'codgeo' existe dans le DataFrame
        if 'codgeo' in df.columns:
            # On convertit toutes les valeurs en minuscules pour ignorer la casse
            valeurs_manquantes = df.loc[~df['codgeo'].str.lower().isin(codes_geo_2024['COM'].str.lower()), 'codgeo']
            
            if not valeurs_manquantes.empty:
                print(f"❌ Valeurs manquantes pour '{nom_sans_extension}' dans 'codes_geo_2024' :")
                print(valeurs_manquantes)
                print(f"✅ DataFrame associé : {nom_sans_extension}")
            else:
                print(f"✅ Toutes les valeurs de 'codgeo' dans '{nom_sans_extension}' sont présentes dans 'codes_geo_2024'.")
        else:
            print(f"⚠️ La colonne 'codgeo' n'existe pas dans le DataFrame '{nom_sans_extension}'.")



❌ Valeurs manquantes pour 'altitude_communes' dans 'codes_geo_2024' :
6153     18131
34756    95282
Name: codgeo, dtype: object
✅ DataFrame associé : altitude_communes
✅ Toutes les valeurs de 'codgeo' dans 'temps_routier_acces_commerces_COM_2024' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'densite_population' sont présentes dans 'codes_geo_2024'.
❌ Valeurs manquantes pour 'infrastructures_sportives' dans 'codes_geo_2024' :
670       98613
671       98805
672       98818
673       98818
674       98818
          ...  
325505    98821
325506    98821
325507    98821
325508    98830
325509    98833
Name: codgeo, Length: 2382, dtype: object
✅ DataFrame associé : infrastructures_sportives
✅ Toutes les valeurs de 'codgeo' dans 'proportion_medecins_sup20min_COM_2021' sont présentes dans 'codes_geo_2024'.
❌ Valeurs manquantes pour 'indicateur_population_COM_2015_23' dans 'codes_geo_2024' :
0       1001
1       1002
2       1004
3       1005
4       1006
       

In [75]:
# # Modif du df Educ_COM_2023

# # Définition d'une fonction pour déconcaténer les lignes si plusieurs codes géo sont fusionnés dans une même cellule
# def deconcatener_lignes(df, colonne):
#     nouvelles_lignes = []  # Liste qui contiendra les lignes corrigées
#     lignes_avant_modification = []  # Lignes originales avec concaténation (pour vérification)
#     lignes_apres_modification = []  # Lignes générées après déconcaténation

#     # Conversion des colonnes (sauf celle contenant les codes géo) en numériques
#     for col in df.columns:
#         if col != colonne:
#             df[col] = pd.to_numeric(df[col], errors='coerce')  # Remplace les erreurs par NaN

#     # Parcours de chaque ligne du DataFrame
#     for index, row in df.iterrows():
#         valeur_concatenee = row[colonne]  # Récupère la valeur de la colonne cible (ex : 'codgeo')
#         str_valeur_concatenee = str(valeur_concatenee)  # Convertit en chaîne de caractères
#         n_parts = len(str_valeur_concatenee) // 5  # Nombre de codes géo de 5 chiffres concaténés

#         # Vérifie qu'on a bien une concaténation multiple de 5 caractères (format codgeo)
#         if n_parts > 1 and len(str_valeur_concatenee) % 5 == 0:
#             lignes_avant_modification.append(row.copy())  # Sauvegarde la ligne d'origine

#             # Pour chaque code géo dans la chaîne concaténée
#             for i in range(n_parts):
#                 valeur_partie = str_valeur_concatenee[i * 5: (i + 1) * 5]  # Extrait un code géo (5 chiffres)
#                 nouvelle_ligne = row.copy()  # Copie de la ligne d'origine
#                 nouvelle_ligne[colonne] = valeur_partie  # Remplace par le code géo isolé

#                 # Divise toutes les autres colonnes numériques par le nombre de parties
#                 for col in df.columns:
#                     if col != colonne and pd.notna(row[col]):
#                         nouvelle_ligne[col] = row[col] / n_parts

#                 nouvelles_lignes.append(nouvelle_ligne)  # Ajoute la ligne corrigée
#                 lignes_apres_modification.append(nouvelle_ligne)  # Sauvegarde aussi pour comparaison
#         else:
#             # Si ce n’est pas une concaténation, on garde la ligne telle quelle
#             nouvelles_lignes.append(row)

#     # Recrée un nouveau DataFrame avec les lignes déconcaténées
#     df_deconcatene = pd.DataFrame(nouvelles_lignes)
#     return df_deconcatene, lignes_avant_modification, lignes_apres_modification

# # Application de la fonction sur le DataFrame Educ_COM_2023, en ciblant la colonne 'codgeo'
# Educ_COM_2023_deconcatene, lignes_avant_modification, lignes_apres_modification = deconcatener_lignes(Educ_COM_2023, 'codgeo')

# # Affiche les 5 premières lignes ayant été modifiées (avant)
# print("Lignes avant modification :")
# for ligne in lignes_avant_modification[:5]: 
#     print(ligne)

# # Affiche les 5 premières lignes après correction
# print("\nLignes après modification :")
# for ligne in lignes_apres_modification[:5]: 
#     print(ligne)


In [76]:
#Educ_COM_2023 = Educ_COM_2023_deconcatene

In [77]:
# modif du df indicateur_population_COM_2015_23
indicateur_population_COM_2015_23['codgeo'] = indicateur_population_COM_2015_23['codgeo'].astype(str).str.zfill(5)
indicateur_population_COM_2015_23.head()

codgeo P21_POP       P21_POP0014       P21_POP1529       P21_POP3044  \
0  01001     832  158.552045228214  100.025669125929  152.787222000046   
1  01002     267  57.0462860507837  32.4390934255431  57.2722235904477   
2  01004   14854  2789.75819358206  3081.37526584419  2740.19262511001   
3  01005    1897    367.3348050417  289.213772744191  388.963943262706   
4  01006     113  9.91228070175439  13.8771929824561  19.8245614035088   

        P21_POP4559       P21_POP6074       P21_POP7589        P21_POP90P  \
0  198.071133518886  149.720217782771  67.0983888134989  5.74532353065543   
1  58.1015060435166  40.5047400920238   19.768433507588    1.867717290097   
2  2632.56587609418  2269.03322839783  1152.13770206965  188.937108902086   
3  415.518703073391  301.217071564669  123.672439577029  11.0792647363145   
4  23.7894736842105  37.6666666666667  7.92982456140351                 0   

           P21_POPH         P21_H0014         P21_H1529         P21_H3044  \
0  423.485016885463  92.9355275683221  47.6380185861432  74.8649592599333   
1  139.890359491632  27.0105124263677  22.3473585071835  24.7401578867594   
2  7125.78010352671  1437.83407401559   1546.0948157482   1386.7882735311   
3  950.047549047935  192.533972330643  143.965257406244  186.671926745086   
4  59.4736842105263  4.95614035087719  7.92982456140351  11.8947368421053   

          P21_H4559         P21_H6074         P21_H7589           P21_H90P  \
0  99.5791105400233  72.9684366139067  32.6179924364153   2.88097188071922   
1  32.7870782781525  21.7130930371851  10.3439914417015  0.948167914282093   
2  1261.03800390197  1000.31179996331  451.303694393293    42.409441973249   
3  209.211074384253  156.367896601423  58.0018065368516   3.29561504343298   
4  12.8859649122807  18.8333333333333  2.97368421052632                  0   

          P21_H0019         P21_H2064          P21_H65P          P21_POPF  \
0  117.365917200565  229.367683115532  76.7514165693656  408.514983114537   
1  40.0980279107553   73.558433904117  26.2338976767596  127.109640508368   
2  2005.81529621262  4062.69026936215  1057.27453795194  7728.21989647329   
3   239.15699748735  558.195808360328  152.694743200257  946.952450952065   
4  6.93859649122807  39.6491228070175  12.8859649122807  53.5263157894737   

          P21_F0014         P21_F1529         P21_F3044         P21_F4559  \
0   65.616517659892  52.3876505397854   77.922262740113  98.4920229788626   
1  30.0357736244161  10.0917349183597  32.5320657036883   25.314427765364   
2  1351.92411956647  1535.28045009598  1353.40435157891  1371.52787219222   
3  174.800832711057  145.248515337947   202.29201651762  206.307628689138   
4  4.95614035087719  5.94736842105263  7.92982456140351  10.9035087719298   

          P21_F6074         P21_F7589           P21_F90P         P21_F0019  \
0  76.7517811688641  34.4803963770836    2.8643516499362  88.9672402899772   
1  18.7916470548387  9.42444206588656  0.919549375814906  37.1631311489844   
2  1268.72142843452  700.834007676362   146.527666928837  1834.03176829307   
3  144.849174963246   65.670633040177   7.78364969288149  221.736061883218   
4  18.8333333333333  4.95614035087719                  0  6.93859649122807   

          P21_F2064          P21_F65P        P21_POP01P  P21_POP01P_IRAN1  \
0  231.363412863245  88.1843299613141  819.279087659675   737.78524450504   
1  69.2585264759809  20.6879828834029  262.998008651787  255.201536574107   
2  4242.01239720769  1652.17573097253  14666.7126170845  12871.2685990156   
3  563.555817844147    161.6605712247  1870.88692001331  1651.79275172059   
4  27.7543859649123  18.8333333333333  111.017543859649  96.1491228070175   

    P21_POP01P_IRAN2  P21_POP01P_IRAN3  P21_POP01P_IRAN4  P21_POP01P_IRAN5  \
0    6.7141026840462  49.4990321965268  19.4514105901707  5.82929768389106   
1  0.919549375814906  4.03241895901838                 0  2.84450374284628   
2   635.676323007423  624.713990425072  226.047603519787  209.65539112

In [78]:
# modif de infrastructures_sportives

# Suppression des lignes où 'codes_geo_2024' commence par '98'
infrastructures_sportives = infrastructures_sportives[~infrastructures_sportives['codgeo'].astype(str).str.startswith('98')]
infrastructures_sportives = infrastructures_sportives[~infrastructures_sportives['codgeo'].astype(str).str.startswith('97')]
infrastructures_sportives['codgeo'] = infrastructures_sportives['codgeo'].astype(str).str.zfill(5)
infrastructures_sportives = infrastructures_sportives[~infrastructures_sportives['codgeo'].astype(str).str.startswith('00nan')]


In [79]:
# modif de StockEtablissement_utf8
StockEtablissement_utf8 = StockEtablissement_utf8[~StockEtablissement_utf8['codgeo'].astype(str).str.startswith('98')]
StockEtablissement_utf8 = StockEtablissement_utf8[~StockEtablissement_utf8['codgeo'].astype(str).str.startswith('97')]
StockEtablissement_utf8 = StockEtablissement_utf8[StockEtablissement_utf8['codgeo'].notna() &  (StockEtablissement_utf8['codgeo'].astype(str).str.strip() != '')]
StockEtablissement_utf8['codgeo'] = StockEtablissement_utf8['codgeo'].replace("21551", "21084")

# Filtrer les établissements actifs (etatAdministratifEtablissement == 'A')
StockEtablissement_utf8 = StockEtablissement_utf8[StockEtablissement_utf8['etatAdministratifEtablissement'] == 'A']
StockEtablissement_utf8 = StockEtablissement_utf8.reset_index(drop=True)



In [80]:
# modif de age_insee
age_insee = age_insee[~age_insee['codgeo'].astype(str).str.startswith('98')]
age_insee = age_insee[~age_insee['codgeo'].astype(str).str.startswith('97')]
age_insee['codgeo'] = age_insee['codgeo'].astype(str).str.zfill(5)
age_insee = age_insee[~age_insee['codgeo'].astype(str).str.startswith('00nan')]

In [81]:
# modif de impots_communes
# Exclure les DOM-TOM 98 et 97
impots_communes = impots_communes[~impots_communes['codgeo'].astype(str).str.startswith('98')]
impots_communes = impots_communes[~impots_communes['codgeo'].astype(str).str.startswith('97')]

# Uniformiser la longueur à 5 caractères (zéro-padding à gauche)
impots_communes['codgeo'] = impots_communes['codgeo'].astype(str).str.zfill(5)

# Supprimer les éventuels '00nan'
impots_communes = impots_communes[~impots_communes['codgeo'].astype(str).str.startswith('00nan')]


In [82]:

table_passage_geo2003_geo2024 = pd.read_excel(
    "insee_millesime_importation/table_passage_geo2003_geo2024.xlsx",
    header=5,
    engine="openpyxl")

# Afficher les premières lignes pour vérifier
print(table_passage_geo2003_geo2024.head())


  CODGEO_INI CODGEO_2024              LIBGEO_2024
0      01001       01001  L'Abergement-Clémenciat
1      01002       01002    L'Abergement-de-Varey
2      01004       01004        Ambérieu-en-Bugey
3      01005       01005      Ambérieux-en-Dombes
4      01006       01006                  Ambléon


In [83]:
# identifier les valeurs manquantes
valeurs_manquantes = StockEtablissement_utf8.loc[~StockEtablissement_utf8['codgeo'].astype(str).str.lower().isin(codes_geo_2024['COM'].astype(str).str.lower()),'codgeo'].drop_duplicates()

# vérifier leur présence dans table_passage_geo2003_geo2024
resultats = pd.DataFrame({'codgeo': valeurs_manquantes})
resultats['trouve_dans_passage'] = resultats['codgeo'].astype(str).isin(table_passage_geo2003_geo2024['CODGEO_INI'].astype(str)).map({True: 'OUI', False: 'NON'})

non_trouves = resultats[resultats['trouve_dans_passage'] == 'NON']
print(" Valeurs non trouvées dans table_passage_geo2003_geo2024 :")
print(non_trouves)

# filtrer les correspondances dans la table de passage
correspondances = table_passage_geo2003_geo2024[
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str).isin(valeurs_manquantes)
][['CODGEO_INI', 'CODGEO_2024']]

# appliquer les remplacements dans StockEtablissement_utf8
for _, row in correspondances.iterrows():
    StockEtablissement_utf8.loc[
        StockEtablissement_utf8['codgeo'].astype(str) == row['CODGEO_INI'],
        'codgeo'
    ] = row['CODGEO_2024']


 Valeurs non trouvées dans table_passage_geo2003_geo2024 :
Empty DataFrame
Columns: [codgeo, trouve_dans_passage]
Index: []


In [84]:
# meme chose pour altitude_communes
valeurs_manquantes_alt = altitude_communes.loc[
    ~altitude_communes['codgeo'].astype(str).str.lower().isin(
        codes_geo_2024['COM'].astype(str).str.lower()
    ),
    'codgeo'
].drop_duplicates()

resultats_alt = pd.DataFrame({'codgeo': valeurs_manquantes_alt})
resultats_alt['trouve_dans_passage'] = resultats_alt['codgeo'].astype(str).isin(
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str)
).map({True: 'OUI', False: 'NON'})

non_trouves_alt = resultats_alt[resultats_alt['trouve_dans_passage'] == 'NON']
print(" Valeurs non trouvées dans table_passage_geo2003_geo2024 pour altitude_communes :")
print(non_trouves_alt)

correspondances_alt = table_passage_geo2003_geo2024[
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str).isin(valeurs_manquantes_alt.astype(str))
][['CODGEO_INI', 'CODGEO_2024']]

for _, row in correspondances_alt.iterrows():
    altitude_communes.loc[
        altitude_communes['codgeo'].astype(str) == row['CODGEO_INI'],
        'codgeo'
    ] = row['CODGEO_2024']


 Valeurs non trouvées dans table_passage_geo2003_geo2024 pour altitude_communes :
Empty DataFrame
Columns: [codgeo, trouve_dans_passage]
Index: []


In [85]:
# 1. Identifier les valeurs manquantes
valeurs_manquantes = impots_communes.loc[
    ~impots_communes['codgeo'].astype(str).str.lower().isin(codes_geo_2024['COM'].astype(str).str.lower()),
    'codgeo'
].drop_duplicates()

# 2. Vérifier leur présence dans table_passage_geo2003_geo2024
resultats = pd.DataFrame({'codgeo': valeurs_manquantes})
resultats['trouve_dans_passage'] = resultats['codgeo'].astype(str).isin(
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str)
).map({True: 'OUI', False: 'NON'})

non_trouves = resultats[resultats['trouve_dans_passage'] == 'NON']
print("Valeurs non trouvées dans table_passage_geo2003_geo2024 :")
print(non_trouves)

# 3. Filtrer les correspondances dans la table de passage
correspondances = table_passage_geo2003_geo2024[
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str).isin(valeurs_manquantes)
][['CODGEO_INI', 'CODGEO_2024']]

# 4. Appliquer les remplacements dans impots_communes
for _, row in correspondances.iterrows():
    impots_communes.loc[
        impots_communes['codgeo'].astype(str) == row['CODGEO_INI'],
        'codgeo'
    ] = row['CODGEO_2024']


Valeurs non trouvées dans table_passage_geo2003_geo2024 :
Empty DataFrame
Columns: [codgeo, trouve_dans_passage]
Index: []


In [86]:
# --- 1. Identifier les valeurs manquantes ---
valeurs_manquantes = employeurs_salaries.loc[
    ~employeurs_salaries['codgeo'].astype(str).str.lower().isin(codes_geo_2024['COM'].astype(str).str.lower()),
    'codgeo'
].drop_duplicates()

# --- 2. Vérifier leur présence dans table_passage_geo2003_geo2024 ---
resultats = pd.DataFrame({'codgeo': valeurs_manquantes})
resultats['trouve_dans_passage'] = resultats['codgeo'].astype(str).isin(
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str)
).map({True: 'OUI', False: 'NON'})

non_trouves = resultats[resultats['trouve_dans_passage'] == 'NON']
print("Valeurs non trouvées dans table_passage_geo2003_geo2024 :")
print(non_trouves)

# --- 3. Filtrer les correspondances dans la table de passage ---
correspondances = table_passage_geo2003_geo2024[
    table_passage_geo2003_geo2024['CODGEO_INI'].astype(str).isin(valeurs_manquantes)
][['CODGEO_INI', 'CODGEO_2024']]

# --- 4. Appliquer les remplacements dans employeurs_salaries ---
for _, row in correspondances.iterrows():
    employeurs_salaries.loc[
        employeurs_salaries['codgeo'].astype(str) == row['CODGEO_INI'],
        'codgeo'
    ] = row['CODGEO_2024']


Valeurs non trouvées dans table_passage_geo2003_geo2024 :
       codgeo trouve_dans_passage
696320   None                 NON


In [87]:

employeurs_salaries = employeurs_salaries.loc[
    ~(employeurs_salaries['codgeo'].isna() |  # NaN
      employeurs_salaries['codgeo'].isin([None, 0]))  # None ou 0
].copy()

# Vérifier
employeurs_salaries.head()


,Région,Ancienne région,Département,Zone d'emploi,EPCI,Nom commune,Commune,Grand secteur d'activité,Secteur NA17,Secteur NA38,Secteur NA88,APE,Code région,Code ancienne région,Code département,Code zone d'emploi,Code EPCI,Code commune,Code APE,Nombre d'établissements 2006,Effectifs salariés 2006,Nombre d'établissements 2007,Effectifs salariés 2007,Nombre d'établissements 2008,Effectifs salariés 2008,Nombre d'établissements 2009,Effectifs salariés 2009,Nombre d'établissements 2010,Effectifs salariés 2010,Nombre d'établissements 2011,Effectifs salariés 2011,Nombre d'établissements 2012,Effectifs salariés 2012,Nombre d'établissements 2013,Effectifs salariés 2013,Nombre d'établissements 2014,Effectifs salariés 2014,Nombre d'établissements 2015,Effectifs salariés 2015,Nombre d'établissements 2016,Effectifs salariés 2016,Nombre d'établissements 2017,Effectifs salariés 2017,Nombre d'établissements 2018,Effectifs salariés 2018,Nombre d'établissements 2019,Effectifs salariés 2019,Nombre d'établissements 2020,Effectifs salariés 2020,Nombre d'établissements 2021,Effectifs salariés 2021,Nombre d'établissements 2022,Effectifs salariés 2022,Nombre d'établissements 2023,Effectifs salariés 2023,Nombre d'établissements 2024,Effectifs salariés 2024,codgeo,Libelle
0,Centre-Val de Loire,Centre,Indre-et-Loire,Tours,Communauté de communes Touraine Val de Vienne,Saint-Épain,37216 Saint-Épain,GS5 Autres services marchands hors intérim,MN Activités scientifiques et techniques ; sou...,NZ Activités de services administratifs et de ...,80 Enquêtes et sécurité,8010Z Activités de sécurité privée,24,24,37,2413,200072668,37216,8010Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37216,Saint-Épain
1,Centre-Val de Loire,Centre,Indre-et-Loire,Tours,Tours Métropole Val de Loire,Saint-Genouph,37219 Saint-Genouph,GS2 Construction,FZ Construction,FZ Construction,43 Travaux de construction spécialisés,4334Z Travaux de peinture et vitrerie,24,24,37,2413,243700754,37219,4334Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,2,1,3,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37219,Saint-Genouph
2,Centre-Val de Loire,Centre,Indre-et-Loire,Tours,Tours Métropole Val de Loire,Saint-Genouph,37219 Saint-Genouph,GS5 Autres services marchands hors intérim,MN Activités scientifiques et techniques ; sou...,"MA Activités juridiques, de conseil et d'ingén...",71 Activités d'architecture et d'ingénierie ; ...,7112A Activité des géomètres,24,24,37,2413,243700754,37219,7112A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1,4,37219,Saint-Genouph
3,Centre-Val de Loire,Centre,Indre-et-Loire,Chinon,"Communauté de communes Chinon, Vienne et Loire",Saint-Germain-sur-Vienne,37220 Saint-Germain-sur-Vienne,GS2 Construction,FZ Construction,FZ Construction,43 Travaux de construction spécialisés,4391B Travaux de couverture par éléments,24,24,37,2406,200043081,37220,4391B,NaN,NaN,1,1,NaN,NaN,1,1,1,3,1,2,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,37220,Saint-Germain-sur-Vienne
4,Centre-Val de Loire,Centre,Indre-et-Loire,Chinon,"Communauté de communes Chinon, Vienne et Loire",Saint-Germain-sur-Vienne,37220 Saint-Germain-sur-Vienne,GS5 Autres services marchands hors intérim,MN Activités scientifiques et techniques ; sou...,NZ Activités de services administratifs et de ...,82 Activités administratives et autres activit...,8299Z Autres activités de soutien aux entrepri...,24,24,37,2406,200043081,37220,8299Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne


In [88]:
# Vérification de l'existence des valeurs de 'codgeo' dans 'COM' de 'codes_geo_2024' en ignorant la casse
for fichier in fichiers:
    nom = os.path.basename(fichier)
    nom_sans_extension = os.path.splitext(nom)[0]  # Extrait le nom sans extension

    if nom_sans_extension in globals():  # Vérifie que le DataFrame est bien chargé dans l'environnement
        df = globals()[nom_sans_extension]
        
        # Vérifie si la colonne 'codgeo' existe dans le DataFrame
        if 'codgeo' in df.columns:
            # On convertit toutes les valeurs en minuscules pour ignorer la casse
            valeurs_manquantes = df.loc[~df['codgeo'].str.lower().isin(codes_geo_2024['COM'].str.lower()), 'codgeo']
            
            if not valeurs_manquantes.empty:
                print(f"❌ Valeurs manquantes pour '{nom_sans_extension}' dans 'codes_geo_2024' :")
                print(valeurs_manquantes)
                print(f"✅ DataFrame associé : {nom_sans_extension}")
            else:
                print(f"✅ Toutes les valeurs de 'codgeo' dans '{nom_sans_extension}' sont présentes dans 'codes_geo_2024'.")
        else:
            print(f"⚠️ La colonne 'codgeo' n'existe pas dans le DataFrame '{nom_sans_extension}'.")



✅ Toutes les valeurs de 'codgeo' dans 'altitude_communes' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'temps_routier_acces_commerces_COM_2024' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'densite_population' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'infrastructures_sportives' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'proportion_medecins_sup20min_COM_2021' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'indicateur_population_COM_2015_23' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'employeurs_salaries' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'tourisme_capacite' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'offre_sante_COM_2023' sont présentes dans 'codes_geo_2024'.
✅ Toutes les valeurs de 'codgeo' dans 'impots_communes' sont prés

In [89]:
# modif de densite pop
densite_population = densite_population[densite_population['an'] == 2021]
densite_population = densite_population.drop(columns=['an', 'libgeo'])
densite_population = densite_population.rename(columns={'dens_pop': 'densite_population'})

densite_population.head()

,codgeo,densite_population
7,01001,52.33
15,01002,29.02
23,01004,603.82
31,01005,119.31
39,01006,19.15


In [90]:
# garder uniquement les colonnes utiles
df_effectifs = employeurs_salaries[['codgeo', 'Effectifs salariés 2024']].copy()

# convertir en numérique (force les erreurs en NaN si besoin)
df_effectifs['Effectifs salariés 2024'] = pd.to_numeric(df_effectifs['Effectifs salariés 2024'], errors='coerce')
df_effectifs = df_effectifs.rename(columns={'Effectifs salariés 2024': 'effectif_salaries'})

# regrouper par commune et sommer
employeurs_salaries = df_effectifs.groupby('codgeo', as_index=False)['effectif_salaries'].sum()

print(employeurs_salaries.head())


  codgeo  effectif_salaries
0  01001               19.0
1  01002                6.0
2  01004             4277.0
3  01005              150.0
4  01007              399.0


In [91]:
# modif de impots communes

colonnes_a_garder = [
    "codgeo",  # identifiant géographique
    "FNB - COMMUNE / MONTANT REEL",
    "FB - COMMUNE / MONTANT REEL",
    "TH - COMMUNE / MONTANT REEL COMMUNAL AU PROFIT DE LA COMMUNE",
    "CFE - COMMUNE / PRODUIT REEL NET"
]


impots_communes = impots_communes[colonnes_a_garder]

impots_communes["impots_moyen"] = (
    impots_communes["FNB - COMMUNE / MONTANT REEL"] +
    impots_communes["FB - COMMUNE / MONTANT REEL"] +
    impots_communes["TH - COMMUNE / MONTANT REEL COMMUNAL AU PROFIT DE LA COMMUNE"] +
    impots_communes["CFE - COMMUNE / PRODUIT REEL NET"]
)

colonnes_a_supprimer = [
    "FNB - COMMUNE / MONTANT REEL",
    "FB - COMMUNE / MONTANT REEL",
    "TH - COMMUNE / MONTANT REEL COMMUNAL AU PROFIT DE LA COMMUNE",
    "CFE - COMMUNE / PRODUIT REEL NET"
]

impots_communes.drop(columns=colonnes_a_supprimer, inplace=True)

impots_communes.head()


,codgeo,impots_moyen
0,01001,178595.0
1,01002,73140.0
2,01004,7597266.0
3,01005,537479.0
4,01006,32722.0


In [92]:
# Filtrer d'abord
activities_to_keep = ['I551', 'I552', 'I552A', 'I552B', 'I552C', 'I553']
tourisme_capacite_filtered = tourisme_capacite[
    (tourisme_capacite['ACTIVITY'].isin(activities_to_keep)) &
    (tourisme_capacite['L_STAY'] == '_T') &
    (tourisme_capacite['GEO_OBJECT'].str.startswith('COM'))
].copy()

# Convertir OBS_VALUE en numérique
tourisme_capacite_filtered['OBS_VALUE'] = pd.to_numeric(
    tourisme_capacite_filtered['OBS_VALUE'], errors='coerce'
)

# Pivot avec somme réelle
tourisme_capacite = (
    tourisme_capacite_filtered
    .pivot_table(
        index='codgeo',
        columns='TOUR_MEASURE',
        values='OBS_VALUE',
        aggfunc='sum',
        fill_value=0
    )
    .rename(columns={
        'PLACE': 'nb_de_places_hebergements_tourisme',
        'UNIT_LOC': 'nb_dhebergements_tourisme'
    })
    .reset_index()[['codgeo','nb_de_places_hebergements_tourisme','nb_dhebergements_tourisme']]
)

tourisme_capacite.head()


TOUR_MEASURE,codgeo,nb_de_places_hebergements_tourisme,nb_dhebergements_tourisme
0,01001,0,0
1,01002,0,0
2,01004,100,4
3,01005,184,6
4,01006,0,0


In [93]:
# modifications de nb_emplois

# année 2021 
nb_emplois = nb_emplois[nb_emplois['an'] == 2021]
nb_emplois['nb_emplois'] = nb_emplois['p_emplt']
nb_emplois = nb_emplois.drop(columns=['an','libgeo','p_emplt'])
nb_emplois.head()

,codgeo,nb_emplois
6,01001,75.0
13,01002,29.0
20,01004,8152.0
27,01005,261.0
34,01006,5.0


In [94]:

pd.set_option('display.max_columns', None)
# Affiche les 5 premières lignes avec toutes les colonnes visibles
StockEtablissement_utf8.head()


,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,dernierNumeroVoieEtablissement,indiceRepetitionDernierNumeroVoieEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codgeo,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,identifiantAdresseEtablissement,coordonneeLambertAbscisseEtablissement,coordonneeLambertOrdonneeEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
0,000325175,00065,00032517500065,O,2018-02-07,NaN,NaN,NaN,2024-03-30T05:28:55,true,1,NaN,51,NaN,NaN,NaN,RUE,MARX DORMOY,13004,MARSEILLE,NaN,NaN,13204,NaN,NaN,NaN,NaN,132045859_B,894569.8467557925,6247518.1219648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-07,A,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N
1,005420021,00056,00542002100056,O,2009-12-23,NaN,NaN,NaN,2025-01-27T02:51:45,true,1,NaN,21,B,NaN,NaN,BOULEVARD,DES PRES,80100,ABBEVILLE,NaN,NaN,80001,NaN,NaN,NaN,NaN,800011770_B,616393.5745431304,7000826.96739121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-12-23,A,NaN,NaN,NaN,NaN,46.69B,NAFRev2,O
2,005420120,00015,00542012000015,O,1989-01-27,01,2022,NaN,2024-03-30T03:44:04,false,4,NaN,NaN,NaN,NaN,NaN,RUE,DE LA FONTAINE,80120,RUE,NaN,NaN,80688,NaN,NaN,NaN,NaN,806880420_B,605361.4086290048,7020443.360403413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-01-01,A,NaN,NaN,NaN,NaN,10.81Z,NAFRev2,O
3,005420120,00023,00542012000023,O,1900-01-01,NaN,NaN,NaN,2024-03-30T03:44:04,false,5,NaN,12,NaN,NaN,NaN,ROUTE,DE MONTREUIL,62180,RANG-DU-FLIERS,NaN,NaN,62688,NaN,NaN,NaN,NaN,626880420_B,603908.6742656698,7036437.727362791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-30,A,NaN,NaN,NaN,NaN,52.10B,NAFRev2,N
4,005420120,00031,00542012000031,O,1900-01-01,03,2022,NaN,2025-02-06T02:30:46,true,6,NaN,NaN,NaN,NaN,NaN,ROUTE,NATIONALE,62140,MARCONNELLE,NaN,NaN,62550,NaN,NaN,NaN,NaN,625500050_B,629760.27887038,7031222.801711981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-23,A,NaN,NaN,NaN,NaN,70.10Z,NAFRev2,O


In [95]:
# Charger la table de correspondance codes - intitulés
entreprises_codes_activites = pd.read_excel("facteurs/documentation_annexe/entreprises_codes_activites.xls")

# Nettoyer les noms de colonnes
entreprises_codes_activites.columns = entreprises_codes_activites.columns.str.strip()

# Renommer la colonne d’intitulés pour plus de clarté
entreprises_codes_activites.rename(columns={
    'Intitulés de la  NAF rév. 2, version finale': 'Intitule_NAF'
}, inplace=True)

# Extraire les codes uniques de ton DataFrame initial
codes_activites = StockEtablissement_utf8['activitePrincipaleEtablissement'].dropna().unique()

# Filtrer la table des activités pour ne garder que celles correspondant à tes codes
intitules_uniques = entreprises_codes_activites[entreprises_codes_activites['Code'].isin(codes_activites)]

# Récupérer uniquement la colonne des intitulés
liste_intitules = intitules_uniques['Intitule_NAF'].unique()

# Afficher ou trier pour plus de lisibilité
liste_intitules = sorted(liste_intitules)
for intitule in liste_intitules:
    print(intitule)


Accueil de jeunes enfants
Accueil ou accompagnement sans hébergement d’adultes handicapés ou de  personnes âgées
Accueil ou accompagnement sans hébergement d’enfants handicapés
Action sociale sans hébergement n.c.a.
Activité des géomètres
Activité des médecins généralistes
Activité des économistes de la construction
Activités chirurgicales
Activités combinées de soutien lié aux bâtiments 
Activités comptables
Activités d'architecture 
Activités d'enquête
Activités de banque centrale
Activités de centres d'appels
Activités de clubs de sports
Activités de conditionnement
Activités de poste dans le cadre d'une obligation de service universel 
Activités de pré-presse 
Activités de radiodiagnostic et de radiothérapie
Activités de santé humaine non classées ailleurs
Activités de soutien au spectacle vivant
Activités de soutien aux autres industries extractives 
Activités de soutien aux cultures
Activités de soutien à l'enseignement
Activités de soutien à l'extraction d'hydrocarbures
Activité

In [96]:
# modifications de StockEtablissement

# Filtrer les établissements actifs (etatAdministratifEtablissement == 'A')
StockEtablissement_utf8 = StockEtablissement_utf8[StockEtablissement_utf8['etatAdministratifEtablissement'] == 'A']
StockEtablissement_utf8 = StockEtablissement_utf8.reset_index(drop=True)

# Garder le codgeo ainsi que activitePrincipaleEtablissement
StockEtablissement_utf8 = StockEtablissement_utf8[['codgeo', 'activitePrincipaleEtablissement']]

# importer entreprises_codes_activités
entreprises_codes_activites = pd.read_excel("facteurs/documentation_annexe/entreprises_codes_activites.xls")

# renommage des colonnes dans entreprises_codes_activites
entreprises_codes_activites.columns = entreprises_codes_activites.columns.str.strip()
entreprises_codes_activites.rename(columns={
    'Intitulés de la  NAF rév. 2, version finale': 'Intitule_NAF'
}, inplace=True)

# Fusion des df
StockEtablissement_utf8 = StockEtablissement_utf8.merge(
    entreprises_codes_activites[['Code', 'Intitule_NAF']],
    how='left',
    left_on='activitePrincipaleEtablissement',
    right_on='Code'
)

# Remplacement des codes par les intitulés
StockEtablissement_utf8['activitePrincipaleEtablissement'] = StockEtablissement_utf8['Intitule_NAF']

# Suppression des colonnes temporaires si elles existent
for col in ['Code', 'Intitule_NAF']:
    if col in StockEtablissement_utf8.columns:
        StockEtablissement_utf8.drop(columns=col, inplace=True)

# Création du tableeau croisé
pivot_table = pd.pivot_table(
    StockEtablissement_utf8,
    index='codgeo',
    columns='activitePrincipaleEtablissement',
    aggfunc='size',
    fill_value=0
)

# Liste des intitulés à conserver
# intitules_a_conserver = [
#     'AGRICULTURE, SYLVICULTURE ET PÊCHE',
#     'Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques',
#     'Élevage de vaches laitières',
#     'Élevage de porcins',
#     'Élevage de volailles',
#     'Extraction de pétrole brut',
#     "Extraction des minéraux chimiques et d'engrais minéraux",
#     'Transformation et conservation de la viande de boucherie',
#     'Enrichissement et retraitement de matières nucléaires',
#     'Fabrication de pesticides et d’autres produits agrochimiques',
#     'Fabrication de produits pharmaceutiques de base',
#     "Production d'électricité",
#     'Traitement et élimination des déchets non dangereux',
#     'Traitement et élimination des déchets dangereux',
#     "Captage, traitement et distribution d'eau",
#     'Collecte et traitement des eaux usées',
#     'Élaboration et transformation de matières nucléaires'
# ]

intitules_a_conserver = [
    # Agriculture & élevage
    'AGRICULTURE, SYLVICULTURE ET PÊCHE',
    'Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques',
    'Élevage de vaches laitières',
    'Élevage de porcins',
    'Élevage de volailles',
    'Culture de céréales (à l\'exception du riz), de légumineuses et de graines oléagineuses',
    'Culture du riz',
    'Culture de légumes, de melons, de racines et de tubercules',
    'Culture de la canne à sucre',
    'Culture du tabac',
    'Culture de plantes à fibres',
    'Autres cultures non permanentes',
    'Culture de la vigne',
    'Culture de fruits tropicaux et subtropicaux',
    'Culture d\'agrumes',
    'Culture de fruits à pépins et à noyau',
    'Culture d\'autres fruits d\'arbres ou d\'arbustes et de fruits à coque',
    'Culture de fruits oléagineux',
    'Culture de plantes à boissons',
    'Autres cultures permanentes',
    'Exploitation forestière',

    # Extraction
    'Extraction de pétrole brut',
    'Extraction de gaz naturel',
    'Extraction de minerais métalliques',
    'Extraction de minerais de fer',
    'Extraction de minerais de métaux non ferreux',
    'Extraction de minerais d\'uranium et de thorium',
    'Extraction d\'autres minerais de métaux non ferreux',

    # Transformation & conservation
    'Transformation et conservation de la viande de boucherie',
    'Enrichissement et retraitement de matières nucléaires',
    'Fabrication de pesticides et d’autres produits agrochimiques',
    'Fabrication de produits pharmaceutiques de base',
    
    # Pollution / traitement déchets / assainissement
    'Collecte des déchets non dangereux',
    'Collecte des déchets dangereux',
    'Traitement et élimination des déchets non dangereux',
    'Traitement et élimination des déchets dangereux',
    'Dépollution et autres services de gestion des déchets',
    'Captage, traitement et distribution d\'eau',
    'Assainissement',
    'Collecte et traitement des eaux usées',
    'Traitement des eaux usées',
    
    # Industrie lourde, chimie, métallurgie, ciment
    "Fabrication d'autres produits chimiques de base",
    "Fabrication de pesticides et d'autres produits agrochimiques",
    'Fabrication de parfums et de produits pour la toilette',
    "Fabrication de savons, détergents, produits d'entretien et de nettoyage",
    'Fabrication de ciment',
    'Sidérurgie',
    'Fabrication de tubes, de tuyaux, de profilés creux et d\'accessoires correspondants en acier',
    'Laminage à chaud de produits plats en acier',
    'Laminage à froid de produits plats en acier',
    'Profilage et emboutissage à froid',
    'Autres travaux de transformation et finissage des métaux',
    'Métallurgie du fer',
    'Métallurgie des métaux légers',
    'Métallurgie des autres métaux non ferreux',
    'Fabrication de coke et de produits de raffinage du pétrole',
    'Fabrication de carburants gazeux',
    'Élaboration et transformation de matières nucléaires',

    # Production d’électricité fossile
    'Production d\'électricité',
    'Transport d\'électricité',
    'Distribution d\'électricité',

    # Transports routiers
    'Transports routiers de fret de proximité',
    'Autres transports routiers de fret',
    'Transports routiers de voyageurs',

    # Tourisme / Hébergement / Restauration
    'Hôtels et hébergement similaire',
    'Hébergement touristique et autre hébergement de courte durée',
    'Restauration traditionnelle',
    'Restauration rapide',
    "Activités des parcs d'attractions et parcs à thèmes",

    # Services publics
    'Administration publique générale',
    'Administration de la sécurité sociale obligatoire',
    'Activités des organisations associatives liées à la protection sociale',
]


# Filtrer les colonnes existantes dans le tableau croisé
colonnes_existantes = [col for col in intitules_a_conserver if col in pivot_table.columns]

# Appliquer le filtre
StockEtablissement_utf8 = pivot_table[colonnes_existantes]

StockEtablissement_utf8.head()



activitePrincipaleEtablissement,"Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques",Élevage de vaches laitières,Élevage de porcins,Élevage de volailles,"Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses",Culture du riz,"Culture de légumes, de melons, de racines et de tubercules",Culture de la canne à sucre,Culture du tabac,Culture de plantes à fibres,Autres cultures non permanentes,Culture de la vigne,Culture de fruits tropicaux et subtropicaux,Culture d'agrumes,Culture de fruits à pépins et à noyau,Culture d'autres fruits d'arbres ou d'arbustes et de fruits à coque,Culture de fruits oléagineux,Culture de plantes à boissons,Autres cultures permanentes,Exploitation forestière,Extraction de pétrole brut,Extraction de gaz naturel,Extraction de minerais de fer,Extraction de minerais d'uranium et de thorium,Extraction d'autres minerais de métaux non ferreux,Transformation et conservation de la viande de boucherie,Fabrication de pesticides et d’autres produits agrochimiques,Fabrication de produits pharmaceutiques de base,Collecte des déchets non dangereux,Collecte des déchets dangereux,Traitement et élimination des déchets non dangereux,Traitement et élimination des déchets dangereux,Dépollution et autres services de gestion des déchets,"Captage, traitement et distribution d'eau",Collecte et traitement des eaux usées,Fabrication de parfums et de produits pour la toilette,Fabrication de ciment,Sidérurgie,Métallurgie des autres métaux non ferreux,Élaboration et transformation de matières nucléaires,Production d'électricité,Transport d'électricité,Distribution d'électricité,Transports routiers de fret de proximité,Restauration traditionnelle,Activités des parcs d'attractions et parcs à thèmes,Administration publique générale
codgeo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001,0,1,1,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
01002,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
01004,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,3,9,2,6
01005,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,1,0,2,0,1
01006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1


In [97]:
# Convertir la colonne 'apl_mg_m65' en type float
acces_medecin_generaliste_COM_2022['apl_mg_m65'] = acces_medecin_generaliste_COM_2022['apl_mg_m65'].astype(float)
acces_medecin_generaliste_COM_2022['apl_mg_m65'] = acces_medecin_generaliste_COM_2022['apl_mg_m65'].fillna(0)

# Calculer la moyenne de 'apl_mg_m65' selon le champ 'codgeo'
mean_acces_medecin_generaliste = acces_medecin_generaliste_COM_2022.groupby('codgeo')['apl_mg_m65'].mean().reset_index()
mean_acces_medecin_generaliste = mean_acces_medecin_generaliste.rename(columns={'apl_mg_m65': 'acces_medecin_generaliste'})

acces_medecin_generaliste = pd.merge(
    mean_acces_medecin_generaliste,
    acces_medecin_generaliste_COM_2022[['codgeo', 'libgeo']].drop_duplicates(),
    on='codgeo'
)

acces_medecin_generaliste = acces_medecin_generaliste[['codgeo', 'libgeo', 'acces_medecin_generaliste']]
print(acces_medecin_generaliste)
print(acces_medecin_generaliste.describe())


      codgeo                   libgeo  acces_medecin_generaliste
0      01001  L'Abergement-Clémenciat                        1.8
1      01002    L'Abergement-de-Varey                        2.0
2      01004        Ambérieu-en-Bugey                        2.9
3      01005      Ambérieux-en-Dombes                        3.9
4      01006                  Ambléon                        0.8
...      ...                      ...                        ...
34930  97613            M'Tsangamouji                        0.0
34931  97614                 Ouangani                        0.0
34932  97615                 Pamandzi                        0.0
34933  97616                     Sada                        0.0
34934  97617                 Tsingoni                        0.0

[34935 rows x 3 columns]
       acces_medecin_generaliste
count               34935.000000
mean                    2.657100
std                     1.227291
min                     0.000000
25%                     1.900

In [98]:
# altitude
altitude_communes = altitude_communes[["codgeo", "altitude_moyenne"]].copy()

altitude_communes.head()

,codgeo,altitude_moyenne
0,01001,242
1,01002,483
2,01004,379
3,01005,290
4,01006,589


In [99]:
acces_soins_infirmiers_COM_2022['apl_inf_hmep'] = acces_soins_infirmiers_COM_2022['apl_inf_hmep'].astype(float)
acces_soins_infirmiers_COM_2022['apl_inf_hmep'] = acces_soins_infirmiers_COM_2022['apl_inf_hmep'].fillna(0)

# Calculer la moyenne de 'apl_mg_m65' selon le champ 'codgeo'
mean_acces_soins_infirmiers = acces_soins_infirmiers_COM_2022.groupby('codgeo')['apl_inf_hmep'].mean().reset_index()
mean_acces_soins_infirmiers = mean_acces_soins_infirmiers.rename(columns={'apl_inf_hmep': 'acces_soins_infirmiers'})

mean_acces_soins_infirmiers = pd.merge(
    mean_acces_soins_infirmiers,
    acces_soins_infirmiers_COM_2022[['codgeo']].drop_duplicates(),
    on='codgeo'
)

acces_soins_infirmiers = mean_acces_soins_infirmiers[['codgeo', 'acces_soins_infirmiers']]
acces_soins_infirmiers.head()

,codgeo,acces_soins_infirmiers
0,01001,121.5
1,01002,109.2
2,01004,208.5
3,01005,145.8
4,01006,47.0


In [100]:
age_insee_group = age_insee.copy()

# 0 à 10 ans
age_insee_group["age_0_10"] = (
    age_insee[["F0-2","F3-5","F6-10",
               "H0-2","H3-5","H6-10"]].sum(axis=1)
)

# 11 à 17 ans
age_insee_group["age_11_17"] = (
    age_insee[["F11-17","H11-17"]].sum(axis=1)
)

# 18 à 39 ans
age_insee_group["age_18_39"] = (
    age_insee[["F18-24","F25-39","H18-24","H25-39"]].sum(axis=1)
)

# 40 à 79 ans
age_insee_group["age_40_79"] = (
    age_insee[["F40-54","F55-64","F65-79",
               "H40-54","H55-64","H65-79"]].sum(axis=1)
)

# 80 ans et plus
age_insee_group["age_80_plus"] = (
    age_insee[["F80+","H80+"]].sum(axis=1)
)

# On garde uniquement les colonnes utiles
age_insee = age_insee_group[["codgeo", "age_0_10", "age_11_17", "age_18_39", "age_40_79", "age_80_plus"]]
age_insee = age_insee.round(0).astype(int, errors="ignore")

In [101]:
'''
Educ_COM_2023 = Educ_COM_2023.rename(columns={
    'EC_MAT' : 'Nb_ecoles_maternelles', 
    'EC_MAT_REP' : 'Nb_ecoles_maternelles_REP',
    'EC_MAT_REP_P' : 'Nb_ecoles_maternelles_REP+',
    'EC_MAT_PRI':'nb_ecoles_maternelles_privees',
    'EC_ELEM':'nb_ecoles_elementaires', 
    'EC_ELEM_REP':'nb_ecoles_elementaires_rep', 
    'EC_ELEM_REP_P':'nb_ecoles_elementaires_REP+',
    'EC_ELEM_PRI ':'nb_ecoles_elementaires_privees',
    'EC_PRI':'nb_ecoles_primaires',
    'EC_PRI_REP':'nb_ecoles_primaires_REP',
    'EC_PRI_REP_P':'nb_ecoles_primaires_REP+',
    'EC_PRI_PRI':'nb_ecoles_primaires_privees',
    'EFF_ECO':'nb_ecoliers' ,
    'COLL': 'nb_colleges',
    'COLL_REP':'nb_colleges_rep',
    'COLL_REP_P':'colleges_rep+',
    'COLL_PRI':'nb_colleges_prives',
    'BOURSE_LYC':'nb_bourses_lycee',
    'BOURSE_COL':'nb_bourses_college',
    'LYC':'nb_lycees',
    'LYC_GT': 'nb_lycees_generaux_technologiques',
    'LYC_GT_PRI':'nb_lycees_generaux_technologiques_prives',
    'LYC_PRO':'nb_lycees_pro',
    'LYC_PRO_PRI':'nb_lycees_pro_prives',
    'LYC_PO':'nb_lycees_polivalents',
    'LYC_PO_PRI':'nb_lycees_polivalents_prives',
    'TX_NSNE_16A25':'part_16-25_sans_emploi_sans_etudes',
    'TFAV_COLL':'nb_collegiens_parents_tres_favorises',
    'FAV_COLL':'nb_collegiens_parents_favorises',
    'MOY_COLL':'nb_collegiens_parents_moyennement_favorises',
    'DEFA_COLL':'nb_collegiens_parents_defavorises',
    'TFAV_LYC_GT':'nb_lyceens_parents_tres_favorises',
    'FAV_LYC_GT':'nb_lyceens_parents_favorises',
    'MOY_LYC_GT':'nb_lyceens_parents_moyennement_favorises',
    'DEFA_LYC_GT':'nb_lyceens_parents_defavorises',
    'TFAV_LYC_PRO':'nb_lyceens_parents_tres_favorises',
    'FAV_LYC_PRO':'nb_lyceens_parents_favorises',
    'MOY_LYC_PRO':'nb_lyceens_parents_moyennement_favorises',
    'DEFA_LYC_PRO':'nb_lyceens_parents_defavorises'
})

# Liste des colonnes à sélectionner
colonnes_a_selectionner = ['codgeo',
                           'Nb_ecoles_maternelles', 
                           'Nb_ecoles_maternelles_REP', 
                           'Nb_ecoles_maternelles_REP+', 
                           'nb_ecoles_maternelles_privees', 
                           'nb_ecoles_elementaires', 
                           'nb_ecoles_elementaires_rep', 
                           'nb_ecoles_elementaires_REP+', 
                           'nb_ecoles_elementaires_privees',
                           'nb_ecoles_primaires',
                           'nb_ecoles_primaires_REP',
                           'nb_ecoles_primaires_REP+',
                           'nb_ecoles_primaires_privees',
                           'nb_ecoliers',
                           'nb_colleges',
                           'nb_colleges_rep',
                           'colleges_rep+',
                           'nb_colleges_prives',
                           'nb_bourses_lycee',
                           'nb_bourses_college',
                           'nb_lycees',
                           'nb_lycees_generaux_technologiques',
                           'nb_lycees_generaux_technologiques_prives',
                           'nb_lycees_pro',
                           'nb_lycees_pro_prives',
                           'nb_lycees_polivalents',
                           'nb_lycees_polivalents_prives',
                           'part_16-25_sans_emploi_sans_etudes',
                           'nb_collegiens_parents_tres_favorises',
                           'nb_collegiens_parents_favorises',
                           'nb_collegiens_parents_moyennement_favorises',
                           'nb_collegiens_parents_defavorises',
                           'nb_lyceens_parents_tres_favorises',
                           'nb_lyceens_parents_favorises',
                           'nb_lyceens_parents_moyennement_favorises',
                           'nb_lyceens_parents_defavorises']

# Sélectionner les colonnes existantes
colonnes_existantes = [col for col in colonnes_a_selectionner if col in Educ_COM_2023.columns]
Educ_COM = Educ_COM_2023[colonnes_existantes]
colonnes_numeriques = [col for col in colonnes_existantes if col != 'codgeo']

# Replace NaN values with 0 and convert to integers
Educ_COM[colonnes_numeriques] = Educ_COM[colonnes_numeriques].fillna(0).astype(float).astype(int)

print(Educ_COM)
'''

"\nEduc_COM_2023 = Educ_COM_2023.rename(columns={\n    'EC_MAT' : 'Nb_ecoles_maternelles', \n    'EC_MAT_REP' : 'Nb_ecoles_maternelles_REP',\n    'EC_MAT_REP_P' : 'Nb_ecoles_maternelles_REP+',\n    'EC_MAT_PRI':'nb_ecoles_maternelles_privees',\n    'EC_ELEM':'nb_ecoles_elementaires', \n    'EC_ELEM_REP':'nb_ecoles_elementaires_rep', \n    'EC_ELEM_REP_P':'nb_ecoles_elementaires_REP+',\n    'EC_ELEM_PRI ':'nb_ecoles_elementaires_privees',\n    'EC_PRI':'nb_ecoles_primaires',\n    'EC_PRI_REP':'nb_ecoles_primaires_REP',\n    'EC_PRI_REP_P':'nb_ecoles_primaires_REP+',\n    'EC_PRI_PRI':'nb_ecoles_primaires_privees',\n    'EFF_ECO':'nb_ecoliers' ,\n    'COLL': 'nb_colleges',\n    'COLL_REP':'nb_colleges_rep',\n    'COLL_REP_P':'colleges_rep+',\n    'COLL_PRI':'nb_colleges_prives',\n    'BOURSE_LYC':'nb_bourses_lycee',\n    'BOURSE_COL':'nb_bourses_college',\n    'LYC':'nb_lycees',\n    'LYC_GT': 'nb_lycees_generaux_technologiques',\n    'LYC_GT_PRI':'nb_lycees_generaux_technologiques_prive

In [102]:

etablissement_indus_haut_risque_COM = etablissement_indus_haut_risque_COM_2024[etablissement_indus_haut_risque_COM_2024['risque_ind'] == 'seveso_tot']

# Sélectionner les colonnes et renommer 'risque_ind' en 'risque_industrie_selon_seveso'
etablissement_indus_haut_risque_COM = etablissement_indus_haut_risque_COM[['codgeo', 'nb_seveso']].rename(columns={'risque_ind': 'nb_industrie_seveso'})
etablissement_indus_haut_risque_COM['nb_seveso'].astype(float).astype(int)
print(etablissement_indus_haut_risque_COM)

       codgeo  nb_seveso
2       01001          0
5       01002          0
8       01004          0
11      01005          0
14      01006          0
...       ...        ...
104792  97613          0
104795  97614          0
104798  97615          0
104801  97616          0
104804  97617          0

[34935 rows x 2 columns]


In [103]:
# Renommage des colonnes
indicateur_population_COM_2015_23 = indicateur_population_COM_2015_23.rename(columns={
    'P21_POP':'Nb_habitants',
    'ENCBE23': 'nb_creation_entreprises_industrie_23',
    'ENCFZ23': 'nb_creation_entreprises_construction_23',
    'ENCGI23': 'nb_creation_entreprises_transport_hebergement_restauration_23',
    'ENCJZ23': 'nb_creation_entreprises_information_communication_23',
    'ENCKZ23': 'nb_creation_entreprises_finance_asssurance_23',
    'ENCLZ23': 'nb_creation_entreprises_immobilier',
    'ENCMN23': 'nb_creation_entreprises_scientifique_technique_admininistratif_23',
    'ENCOQ23': 'nb_creation_entreprises_public_sante_enseignement_23',
    'ENCRU23': 'nb_creation_entreprises_autre_23',
    'MED21': 'med_du_niveau_de_vie',
    'NBMENFISC21': 'nb_menages_fiscaux',
    'TP6021': 'taux_pauvrete',
    'TP60TOL121': 'taux_pauvrete_proprietaire',
    'TP60TOL221': 'taux_pauvrete_locataire',  # <- correction ici
    'SNHM22': 'moy_salaire_net_horaire',
    'SNHMC22': 'moy_salaire_net_horaire_cadres_sup',
    'SNHMP22': 'moy_salaire_net_horaire_professions_intermediaires',
    'SNHME22': 'moy_salaire_net_horaire_employes',
    'SNHMO22': 'moy_salaire_net_horaire_ouvriers',
    'SNHMF22': 'moy_salaire_femme_net_horaire',
    'SNHMH22': 'moy_salaire_homme_net_horaire'
})

# Colonnes à sélectionner
selected_columns = [
    'codgeo',
    'Nb_habitants',
    'nb_creation_entreprises_industrie_23',
    'nb_creation_entreprises_construction_23',
    'nb_creation_entreprises_transport_hebergement_restauration_23',
    'nb_creation_entreprises_information_communication_23',
    'nb_creation_entreprises_finance_asssurance_23',
    'nb_creation_entreprises_immobilier',
    'nb_creation_entreprises_scientifique_technique_admininistratif_23',
    'nb_creation_entreprises_public_sante_enseignement_23',
    'nb_creation_entreprises_autre_23',
    'med_du_niveau_de_vie',
    'nb_menages_fiscaux',
    'taux_pauvrete',
    'taux_pauvrete_proprietaire',
    'taux_pauvrete_locataire',  # <- correction ici
    'moy_salaire_net_horaire',
    'moy_salaire_net_horaire_cadres_sup',
    'moy_salaire_net_horaire_professions_intermediaires',
    'moy_salaire_net_horaire_employes',
    'moy_salaire_femme_net_horaire',
    'moy_salaire_homme_net_horaire'
]

indicateur_population = indicateur_population_COM_2015_23[selected_columns]

# Conversion en float 
for col in indicateur_population.columns:
    if col != 'codgeo':
        indicateur_population[col] = pd.to_numeric(indicateur_population[col], errors='coerce')

# Affichage
print(indicateur_population.head())


/tmp/ipykernel_551/3518002838.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indicateur_population[col] = pd.to_numeric(indicateur_population[col], errors='coerce')


  codgeo  Nb_habitants  nb_creation_entreprises_industrie_23  \
0  01001         832.0                                   0.0   
1  01002         267.0                                   0.0   
2  01004       14854.0                                   7.0   
3  01005        1897.0                                   1.0   
4  01006         113.0                                   1.0   

   nb_creation_entreprises_construction_23  \
0                                      2.0   
1                                      0.0   
2                                     17.0   
3                                      2.0   
4                                      0.0   

   nb_creation_entreprises_transport_hebergement_restauration_23  \
0                                                0.0               
1                                                1.0               
2                                               61.0               
3                                                4.0              

In [104]:
indice_position_sociale_ecole_COM_2018_23.head()
indice_position_sociale_ecole_COM_2018_23 = indice_position_sociale_ecole_COM_2018_23[indice_position_sociale_ecole_COM_2018_23['secteur_ensgn'].notna()]

indice_position_sociale_ecole_COM = indice_position_sociale_ecole_COM_2018_23.groupby('codgeo')['ips_lycees'].mean().reset_index()
indice_position_sociale_ecole_COM['ips_lycees'] = indice_position_sociale_ecole_COM['ips_lycees'].astype(float)
indice_position_sociale_ecole_COM.head()


,codgeo,ips_lycees
0,01004,110.160000
1,01031,101.540000
2,01033,95.860000
3,01034,104.966667
4,01049,122.860000


In [105]:
nb_decheterie_COM_22 = nb_decheterie_COM_22.drop(columns=['libgeo', 'an',])
nb_decheterie_COM_22['nb_ctr_tri'].astype(float).astype(int)
nb_decheterie_COM_22.head()

,codgeo,nb_ctr_tri
0,01001,0
1,01002,0
2,01004,1
3,01005,0
4,01006,0


In [106]:
nb_exploitations_agri_Bio_COM_2023 = nb_exploitations_agri_Bio_COM_2023.drop(columns=['libgeo', 'an'])

nb_exploitations_agri_Bio_COM_2023['nb_exp'] = nb_exploitations_agri_Bio_COM_2023['nb_exp'].fillna(0)
nb_exploitations_agri_Bio_COM_2023['nb_exp'] = nb_exploitations_agri_Bio_COM_2023['nb_exp'].astype(int)
nb_exploitations_agri_Bio_COM_2023.rename(columns={'nb_exp': 'nb_exploitations_agri_bio'}, inplace=True)
nb_exploitations_agri_Bio_COM_2023.head()


,codgeo,nb_exploitations_agri_bio
0,01001,0
1,01002,1
2,01004,1
3,01005,0
4,01006,0


In [107]:
nb_structures_France_Travail_COM_2023 = nb_structures_France_Travail_COM_2023.drop(columns=['libgeo'])

nb_structures_France_Travail_COM_2023['nb_france_travail'] = nb_structures_France_Travail_COM_2023['nb_france_travail'].fillna(0)
nb_structures_France_Travail_COM_2023['nb_france_travail'] = nb_structures_France_Travail_COM_2023['nb_france_travail'].astype(float).astype(int)

nb_structures_France_Travail_COM_2023.head()


,codgeo,nb_france_travail
0,01001,0
1,01002,0
2,01004,1
3,01005,0
4,01006,0


In [108]:
offre_sante_COM_2023 = offre_sante_COM_2023.drop(columns=['libgeo', 'an',])
offre_sante_COM_2023 = offre_sante_COM_2023.pivot_table(index=['codgeo'], columns='offre_sante', values='nombre').reset_index()

for col in offre_sante_COM_2023.columns:
    if col != 'codgeo':
        offre_sante_COM_2023[col] = offre_sante_COM_2023[col].fillna(0).astype(int)

offre_sante_COM_2023.head()


offre_sante,codgeo,card,chirdent,derm,gastro,gynec,inf,masskine,medg,opht,orl,pedia,pneumo,psychi,psycho,radio,sgf
0,01001,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,01002,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,01004,8,15,0,4,7,32,24,24,3,5,1,1,1,10,23,6
3,01005,0,2,0,0,0,3,2,2,0,0,0,0,0,0,0,0
4,01006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
proportion_medecins_sup20min_COM_2021 = proportion_medecins_sup20min_COM_2021.drop(columns=['libgeo', 'an'])
proportion_medecins_sup20min_COM_2021['part_pop_eloigne_soin'] = proportion_medecins_sup20min_COM_2021['part_pop_eloigne_soin'].astype(float)/100
proportion_medecins_sup20min_COM_2021.head()

,codgeo,part_pop_eloigne_soin
0,01001,0.0
1,01002,0.0
2,01004,0.0
3,01005,0.0
4,01006,0.0


In [110]:
salaire_secteur_COM_2022 = salaire_secteur_COM_2022.drop(columns=['Libellé géographique'])
cols_to_convert = salaire_secteur_COM_2022.columns.difference(['codgeo'])
salaire_secteur_COM_2022[cols_to_convert] = salaire_secteur_COM_2022[cols_to_convert].astype(float)
salaire_secteur_COM_2022.columns = salaire_secteur_COM_2022.columns.str.replace(' ', '_')  # Remplace les espaces par des "_"
salaire_secteur_COM_2022.columns = salaire_secteur_COM_2022.columns.str.replace("_en_2022_(€)", "")
salaire_secteur_COM_2022.columns = salaire_secteur_COM_2022.columns.str.replace('.', '')
salaire_secteur_COM_2022.head()


,codgeo,Salaire_net_horaire_moyen,Salaire_net_hor_Moy_Cadres_sup,Salaire_net_hor_moy_prof_inter,Salaire_net_hor_moy_employés,Salaire_net_hor_moy_ouvriers,Salaire_net_horaire_moyen_F,Salaire_net_hor_moy_F_cadres_sup,Salaire_net_hor_moy_F_prof_inter,Salaire_net_hor_moy_F_employés,Salaire_net_hor_moy_F_ouvriers,Salaire_net_horaire_moyen_H,Salaire_net_hor_moy_H_cadres_sup,Salaire_net_hor_moy_H_prof_inter,Salaire_net_hor_moy_H_employés,Salaire_net_hor_moy_H_ouvriers,Salaire_net_horaire_moyen_18_à_25_ans,Salaire_net_horaire_moyen_26_à_50_ans,Salaire_net_horaire_moyen_plus_de_50_ans,Salaire_net_horaire_moyen_F_18_à_25_ans,Salaire_net_horaire_moyen_F_26_à_50_ans,Salaire_net_horaire_moyen_F_plus_de_50_ans,Salaire_net_horaire_moyen_H_18_à_25_ans,Salaire_net_horaire_moyen_H_26_à_50_ans,Salaire_net_horaire_moyen_H_plus_de_50_ans
0,01004,15.312440,24.969084,16.993865,11.705337,12.362179,13.479085,21.485803,15.016425,11.555528,11.033614,16.451273,26.487448,18.203089,12.101572,12.771241,11.370577,15.491043,17.243640,10.863101,13.720662,14.457493,11.706597,16.549479,19.092391
1,01007,16.237951,25.237836,17.310098,12.657432,12.664244,14.248596,21.026472,15.392349,12.546919,11.382280,17.841462,27.295452,18.919653,13.098365,13.148942,11.234755,16.264425,17.767450,10.704492,14.427844,14.908967,11.727948,17.657295,20.405407
2,01014,14.917202,26.753457,17.207263,12.222352,12.455691,13.151516,19.579961,14.620949,12.561524,11.798639,16.151193,29.992089,18.391474,10.854014,12.770931,11.575799,14.408777,16.888923,10.940382,12.908915,14.345065,11.993264,15.525384,18.504021
3,01024,15.349892,24.153561,16.813791,12.457960,12.955755,13.943065,21.370830,15.033666,12.343133,11.157080,16.303028,25.710001,18.051926,12.919415,13.329430,11.830578,15.352004,16.590929,11.104565,14.116621,14.504951,12.277018,16.187701,18.057664
4,01025,15.535065,24.679005,16.396846,12.512485,12.957097,14.304033,23.369807,14.759745,12.269881,11.344592,16.329973,25.386701,17.534913,13.314054,13.309326,11.209978,15.627295,16.764480,10.594928,14.422166,15.339260,11.613505,16.438346,17.600552


In [111]:
temps_routier_acces_commerces_COM_2024 = temps_routier_acces_commerces_COM_2024[['codgeo', 'temps_acces_centralites']]
temps_routier_acces_commerces_COM_2024 = temps_routier_acces_commerces_COM_2024.fillna(0)
temps_routier_acces_commerces_COM_2024['temps_acces_centralites'] = list(map(float, temps_routier_acces_commerces_COM_2024['temps_acces_centralites']))
temps_routier_acces_commerces_COM_2024.head()


,codgeo,temps_acces_centralites
0,01001,7.3
1,01002,14.2
2,01004,0.0
3,01005,11.4
4,01006,13.6


In [112]:
types_a_conserver = ['Complexe sportif', 'Piscine']

# Forcer codgeo en string dès le départ
infrastructures_sportives['codgeo'] = infrastructures_sportives['codgeo'].astype(str)

# Conserver tous les codgeo existants
codgeo_tous = infrastructures_sportives[['codgeo']].drop_duplicates()

# Séparer les types multiples en lignes distinctes
df_explosé = infrastructures_sportives.assign(
    inst_part_type=infrastructures_sportives['inst_part_type'].str.strip('[]').str.split(',')
).explode('inst_part_type')

# Nettoyage
df_explosé['inst_part_type'] = df_explosé['inst_part_type'].str.strip().str.strip('"')

# Filtrage des types à conserver
df_filtré = df_explosé[df_explosé['inst_part_type'].isin(types_a_conserver)]

# Comptage par codgeo et type
df_groupé = df_filtré.groupby(['codgeo', 'inst_part_type']).size().unstack(fill_value=0).reset_index()
df_groupé.columns.name = None

# Forcer les deux codgeo à être du même type (string)
codgeo_tous['codgeo'] = codgeo_tous['codgeo'].astype(str)
df_groupé['codgeo'] = df_groupé['codgeo'].astype(str)

# Merge en conservant tous les codgeo
infrastructures_sportives = pd.merge(codgeo_tous, df_groupé, on='codgeo', how='left').fillna(0)

# Convertir les colonnes de comptage en entiers
for col in types_a_conserver:
    if col in infrastructures_sportives.columns:
        infrastructures_sportives[col] = infrastructures_sportives[col].astype(int)
    else:
        infrastructures_sportives[col] = 0  # Ajouter la colonne manquante si elle n'est pas présente

# Tri optionnel
infrastructures_sportives = infrastructures_sportives.sort_values('codgeo').reset_index(drop=True)

# Affichage
infrastructures_sportives.head()


,codgeo,Complexe sportif,Piscine
0,01001,0,0
1,01002,0,0
2,01004,21,2
3,01005,0,0
4,01006,0,0


In [130]:
# analyse finale de chacun des df pour vérifier leur nombre de lignes, s'il y a des nan etc.
dataframes = {
    "acces_medecin_generaliste": acces_medecin_generaliste,
    "acces_soins_infirmiers": acces_soins_infirmiers,
    #"Educ_COM": Educ_COM,
    "etablissement_indus_haut_risque_COM": etablissement_indus_haut_risque_COM,
    "indicateur_population": indicateur_population,
    "indice_position_sociale_ecole_COM": indice_position_sociale_ecole_COM,
    "nb_decheterie_COM_22": nb_decheterie_COM_22,
    "nb_exploitations_agri_Bio_COM_2023": nb_exploitations_agri_Bio_COM_2023,
    "nb_structures_France_Travail_COM_2023": nb_structures_France_Travail_COM_2023,
    "offre_sante_COM_2023": offre_sante_COM_2023,
    "proportion_medecins_sup20min_COM_2021": proportion_medecins_sup20min_COM_2021,
    "salaire_secteur_COM_2022": salaire_secteur_COM_2022,
    "temps_routier_acces_commerces_COM_2024": temps_routier_acces_commerces_COM_2024,
    "infrastructures_sportives_COM_2024": infrastructures_sportives,
    "StockEtablissement_utf8":StockEtablissement_utf8,
    "tourisme_capacite":tourisme_capacite,
    "nb_emplois":nb_emplois,
    "age_insee":age_insee,
    "altitude_communes":altitude_communes,
    "employeurs_salaries":employeurs_salaries,
    "densite_population":densite_population,
    "impots_communes":impots_communes,

}

for name, df in dataframes.items():
    nb_lignes = len(df)
    nb_nan = df.isna().sum().sum()
    print(f"{name} : {nb_lignes} lignes, {nb_nan} valeurs manquantes (NaN)")



acces_medecin_generaliste : 34935 lignes, 0 valeurs manquantes (NaN)
acces_soins_infirmiers : 34935 lignes, 0 valeurs manquantes (NaN)
etablissement_indus_haut_risque_COM : 34935 lignes, 0 valeurs manquantes (NaN)
indicateur_population : 34980 lignes, 280216 valeurs manquantes (NaN)
indice_position_sociale_ecole_COM : 1368 lignes, 0 valeurs manquantes (NaN)
nb_decheterie_COM_22 : 34935 lignes, 0 valeurs manquantes (NaN)
nb_exploitations_agri_Bio_COM_2023 : 34935 lignes, 0 valeurs manquantes (NaN)
nb_structures_France_Travail_COM_2023 : 34935 lignes, 0 valeurs manquantes (NaN)
offre_sante_COM_2023 : 34935 lignes, 0 valeurs manquantes (NaN)
proportion_medecins_sup20min_COM_2021 : 34935 lignes, 23 valeurs manquantes (NaN)
salaire_secteur_COM_2022 : 5425 lignes, 48 valeurs manquantes (NaN)
temps_routier_acces_commerces_COM_2024 : 34935 lignes, 0 valeurs manquantes (NaN)
infrastructures_sportives_COM_2024 : 28308 lignes, 0 valeurs manquantes (NaN)
StockEtablissement_utf8 : 34880 lignes, 0 v

In [131]:
factors = acces_medecin_generaliste.merge(acces_soins_infirmiers, on='codgeo', how='outer')\
                                  .merge(etablissement_indus_haut_risque_COM, on='codgeo', how='outer')\
                                  .merge(indicateur_population, on='codgeo', how='outer')\
                                  .merge(indice_position_sociale_ecole_COM, on='codgeo', how='outer')\
                                  .merge(nb_decheterie_COM_22, on='codgeo', how='outer')\
                                  .merge(nb_exploitations_agri_Bio_COM_2023, on='codgeo', how='outer')\
                                  .merge(nb_structures_France_Travail_COM_2023, on='codgeo', how='outer')\
                                  .merge(offre_sante_COM_2023, on='codgeo', how='outer')\
                                  .merge(proportion_medecins_sup20min_COM_2021, on='codgeo', how='outer')\
                                  .merge(salaire_secteur_COM_2022, on='codgeo', how='outer')\
                                  .merge(temps_routier_acces_commerces_COM_2024, on='codgeo', how='outer')\
                                  .merge(infrastructures_sportives, on='codgeo', how='outer')\
                                  .merge(StockEtablissement_utf8, on='codgeo', how='outer')\
                                  .merge(tourisme_capacite, on='codgeo', how='outer')\
                                  .merge(nb_emplois, on='codgeo', how='outer')\
                                  .merge(age_insee, on='codgeo', how='outer')\
                                  .merge(altitude_communes, on='codgeo', how='outer')\
                                  .merge(densite_population, on='codgeo', how='outer')\
                                  .merge(employeurs_salaries, on='codgeo', how='outer')\
                                  .merge(impots_communes, on='codgeo', how='outer')

#.merge(Educ_COM, on='codgeo', how='outer')\

# Afficher le résultat
factors.head()


,codgeo,libgeo,acces_medecin_generaliste,acces_soins_infirmiers,nb_seveso,Nb_habitants,nb_creation_entreprises_industrie_23,nb_creation_entreprises_construction_23,nb_creation_entreprises_transport_hebergement_restauration_23,nb_creation_entreprises_information_communication_23,nb_creation_entreprises_finance_asssurance_23,nb_creation_entreprises_immobilier,nb_creation_entreprises_scientifique_technique_admininistratif_23,nb_creation_entreprises_public_sante_enseignement_23,nb_creation_entreprises_autre_23,med_du_niveau_de_vie,nb_menages_fiscaux,taux_pauvrete,taux_pauvrete_proprietaire,taux_pauvrete_locataire,moy_salaire_net_horaire,moy_salaire_net_horaire_cadres_sup,moy_salaire_net_horaire_professions_intermediaires,moy_salaire_net_horaire_employes,moy_salaire_femme_net_horaire,moy_salaire_homme_net_horaire,ips_lycees,nb_ctr_tri,nb_exploitations_agri_bio,nb_france_travail,card,chirdent,derm,gastro,gynec,inf,masskine,medg,opht,orl,pedia,pneumo,psychi,psycho,radio,sgf,part_pop_eloigne_soin,Salaire_net_horaire_moyen,Salaire_net_hor_Moy_Cadres_sup,Salaire_net_hor_moy_prof_inter,Salaire_net_hor_moy_employés,Salaire_net_hor_moy_ouvriers,Salaire_net_horaire_moyen_F,Salaire_net_hor_moy_F_cadres_sup,Salaire_net_hor_moy_F_prof_inter,Salaire_net_hor_moy_F_employés,Salaire_net_hor_moy_F_ouvriers,Salaire_net_horaire_moyen_H,Salaire_net_hor_moy_H_cadres_sup,Salaire_net_hor_moy_H_prof_inter,Salaire_net_hor_moy_H_employés,Salaire_net_hor_moy_H_ouvriers,Salaire_net_horaire_moyen_18_à_25_ans,Salaire_net_horaire_moyen_26_à_50_ans,Salaire_net_horaire_moyen_plus_de_50_ans,Salaire_net_horaire_moyen_F_18_à_25_ans,Salaire_net_horaire_moyen_F_26_à_50_ans,Salaire_net_horaire_moyen_F_plus_de_50_ans,Salaire_net_horaire_moyen_H_18_à_25_ans,Salaire_net_horaire_moyen_H_26_à_50_ans,Salaire_net_horaire_moyen_H_plus_de_50_ans,temps_acces_centralites,Complexe sportif,Piscine,"Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques",Élevage de vaches laitières,Élevage de porcins,Élevage de volailles,"Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses",Culture du riz,"Culture de légumes, de melons, de racines et de tubercules",Culture de la canne à sucre,Culture du tabac,Culture de plantes à fibres,Autres cultures non permanentes,Culture de la vigne,Culture de fruits tropicaux et subtropicaux,Culture d'agrumes,Culture de fruits à pépins et à noyau,Culture d'autres fruits d'arbres ou d'arbustes et de fruits à coque,Culture de fruits oléagineux,Culture de plantes à boissons,Autres cultures permanentes,Exploitation forestière,Extraction de pétrole brut,Extraction de gaz naturel,Extraction de minerais de fer,Extraction de minerais d'uranium et de thorium,Extraction d'autres minerais de métaux non ferreux,Transformation et conservation de la viande de boucherie,Fabrication de pesticides et d’autres produits agrochimiques,Fabrication de produits pharmaceutiques de base,Collecte des déchets non dangereux,Collecte des déchets dangereux,Traitement et élimination des déchets non dangereux,Traitement et élimination des déchets dangereux,Dépollution et autres services de gestion des déchets,"Captage, traitement et distribution d'eau",Collecte et traitement des eaux usées,Fabrication de parfums et de produits pour la toilette,Fabrication de ciment,Sidérurgie,Métallurgie des autres métaux non ferreux,Élaboration et transformation de matières nucléaires,Production d'électricité,Transport d'électricité,Distribution d'électricité,Transports routiers de fret de proximité,Restauration traditionnelle,Activités des parcs d'attractions et parcs à thèmes,Administration publique générale,nb_de_places_hebergements_tourisme,nb_dhebergements_tourisme,nb_emplois,age_0_10,age_11_17,age_18_39,age_40_79,age_80_plus,altitude_moyenne,densite_population,effectif_salaries,impots_moyen
0,01001,L'Abergement-Clémenciat,1.8,121.5,0.0,832.0,0.0,2.0,0.0,1.0,0.0,0.0,7.0,0.0,3.0,25820.0,346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,

In [132]:
# fais afficher le nb de valeurs nulles
print("Nombre total de lignes :", len(factors))

Nombre total de lignes : 35039


In [133]:
# transformer les NaN en 0
factors = factors.fillna(0)


In [134]:
# faire le summary de chaque colonne

In [135]:
str_cols = factors.select_dtypes(include=['object', 'string']).columns.tolist()
print("Colonnes de type string :", str_cols)

Colonnes de type string : ['codgeo', 'libgeo', 'altitude_moyenne']


In [136]:
exclude_cols = ['codgeo', 'libgeo']

factors_qual = factors.copy()

numeric_cols = [
    col for col in factors_qual.columns
    if col not in exclude_cols and pd.api.types.is_numeric_dtype(factors_qual[col])
]

# Transformation des colonnes numériques
for col in numeric_cols:
    q1 = factors_qual[col].quantile(0.25)
    q2 = factors_qual[col].quantile(0.50)
    q3 = factors_qual[col].quantile(0.75)

    def classify(val):
        if pd.isna(val):
            return None
        elif val == 0:
            return 0
        elif val <= q1:
            return 1
        elif val <= q2:
            return 2
        elif val <= q3:
            return 3
        else:
            return 4

    factors_qual[col] = factors_qual[col].apply(classify)


In [137]:
factors.head()

,codgeo,libgeo,acces_medecin_generaliste,acces_soins_infirmiers,nb_seveso,Nb_habitants,nb_creation_entreprises_industrie_23,nb_creation_entreprises_construction_23,nb_creation_entreprises_transport_hebergement_restauration_23,nb_creation_entreprises_information_communication_23,nb_creation_entreprises_finance_asssurance_23,nb_creation_entreprises_immobilier,nb_creation_entreprises_scientifique_technique_admininistratif_23,nb_creation_entreprises_public_sante_enseignement_23,nb_creation_entreprises_autre_23,med_du_niveau_de_vie,nb_menages_fiscaux,taux_pauvrete,taux_pauvrete_proprietaire,taux_pauvrete_locataire,moy_salaire_net_horaire,moy_salaire_net_horaire_cadres_sup,moy_salaire_net_horaire_professions_intermediaires,moy_salaire_net_horaire_employes,moy_salaire_femme_net_horaire,moy_salaire_homme_net_horaire,ips_lycees,nb_ctr_tri,nb_exploitations_agri_bio,nb_france_travail,card,chirdent,derm,gastro,gynec,inf,masskine,medg,opht,orl,pedia,pneumo,psychi,psycho,radio,sgf,part_pop_eloigne_soin,Salaire_net_horaire_moyen,Salaire_net_hor_Moy_Cadres_sup,Salaire_net_hor_moy_prof_inter,Salaire_net_hor_moy_employés,Salaire_net_hor_moy_ouvriers,Salaire_net_horaire_moyen_F,Salaire_net_hor_moy_F_cadres_sup,Salaire_net_hor_moy_F_prof_inter,Salaire_net_hor_moy_F_employés,Salaire_net_hor_moy_F_ouvriers,Salaire_net_horaire_moyen_H,Salaire_net_hor_moy_H_cadres_sup,Salaire_net_hor_moy_H_prof_inter,Salaire_net_hor_moy_H_employés,Salaire_net_hor_moy_H_ouvriers,Salaire_net_horaire_moyen_18_à_25_ans,Salaire_net_horaire_moyen_26_à_50_ans,Salaire_net_horaire_moyen_plus_de_50_ans,Salaire_net_horaire_moyen_F_18_à_25_ans,Salaire_net_horaire_moyen_F_26_à_50_ans,Salaire_net_horaire_moyen_F_plus_de_50_ans,Salaire_net_horaire_moyen_H_18_à_25_ans,Salaire_net_horaire_moyen_H_26_à_50_ans,Salaire_net_horaire_moyen_H_plus_de_50_ans,temps_acces_centralites,Complexe sportif,Piscine,"Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques",Élevage de vaches laitières,Élevage de porcins,Élevage de volailles,"Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses",Culture du riz,"Culture de légumes, de melons, de racines et de tubercules",Culture de la canne à sucre,Culture du tabac,Culture de plantes à fibres,Autres cultures non permanentes,Culture de la vigne,Culture de fruits tropicaux et subtropicaux,Culture d'agrumes,Culture de fruits à pépins et à noyau,Culture d'autres fruits d'arbres ou d'arbustes et de fruits à coque,Culture de fruits oléagineux,Culture de plantes à boissons,Autres cultures permanentes,Exploitation forestière,Extraction de pétrole brut,Extraction de gaz naturel,Extraction de minerais de fer,Extraction de minerais d'uranium et de thorium,Extraction d'autres minerais de métaux non ferreux,Transformation et conservation de la viande de boucherie,Fabrication de pesticides et d’autres produits agrochimiques,Fabrication de produits pharmaceutiques de base,Collecte des déchets non dangereux,Collecte des déchets dangereux,Traitement et élimination des déchets non dangereux,Traitement et élimination des déchets dangereux,Dépollution et autres services de gestion des déchets,"Captage, traitement et distribution d'eau",Collecte et traitement des eaux usées,Fabrication de parfums et de produits pour la toilette,Fabrication de ciment,Sidérurgie,Métallurgie des autres métaux non ferreux,Élaboration et transformation de matières nucléaires,Production d'électricité,Transport d'électricité,Distribution d'électricité,Transports routiers de fret de proximité,Restauration traditionnelle,Activités des parcs d'attractions et parcs à thèmes,Administration publique générale,nb_de_places_hebergements_tourisme,nb_dhebergements_tourisme,nb_emplois,age_0_10,age_11_17,age_18_39,age_40_79,age_80_plus,altitude_moyenne,densite_population,effectif_salaries,impots_moyen
0,01001,L'Abergement-Clémenciat,1.8,121.5,0.0,832.0,0.0,2.0,0.0,1.0,0.0,0.0,7.0,0.0,3.0,25820.0,346.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00

In [138]:
factors_qual.head()

,codgeo,libgeo,acces_medecin_generaliste,acces_soins_infirmiers,nb_seveso,Nb_habitants,nb_creation_entreprises_industrie_23,nb_creation_entreprises_construction_23,nb_creation_entreprises_transport_hebergement_restauration_23,nb_creation_entreprises_information_communication_23,nb_creation_entreprises_finance_asssurance_23,nb_creation_entreprises_immobilier,nb_creation_entreprises_scientifique_technique_admininistratif_23,nb_creation_entreprises_public_sante_enseignement_23,nb_creation_entreprises_autre_23,med_du_niveau_de_vie,nb_menages_fiscaux,taux_pauvrete,taux_pauvrete_proprietaire,taux_pauvrete_locataire,moy_salaire_net_horaire,moy_salaire_net_horaire_cadres_sup,moy_salaire_net_horaire_professions_intermediaires,moy_salaire_net_horaire_employes,moy_salaire_femme_net_horaire,moy_salaire_homme_net_horaire,ips_lycees,nb_ctr_tri,nb_exploitations_agri_bio,nb_france_travail,card,chirdent,derm,gastro,gynec,inf,masskine,medg,opht,orl,pedia,pneumo,psychi,psycho,radio,sgf,part_pop_eloigne_soin,Salaire_net_horaire_moyen,Salaire_net_hor_Moy_Cadres_sup,Salaire_net_hor_moy_prof_inter,Salaire_net_hor_moy_employés,Salaire_net_hor_moy_ouvriers,Salaire_net_horaire_moyen_F,Salaire_net_hor_moy_F_cadres_sup,Salaire_net_hor_moy_F_prof_inter,Salaire_net_hor_moy_F_employés,Salaire_net_hor_moy_F_ouvriers,Salaire_net_horaire_moyen_H,Salaire_net_hor_moy_H_cadres_sup,Salaire_net_hor_moy_H_prof_inter,Salaire_net_hor_moy_H_employés,Salaire_net_hor_moy_H_ouvriers,Salaire_net_horaire_moyen_18_à_25_ans,Salaire_net_horaire_moyen_26_à_50_ans,Salaire_net_horaire_moyen_plus_de_50_ans,Salaire_net_horaire_moyen_F_18_à_25_ans,Salaire_net_horaire_moyen_F_26_à_50_ans,Salaire_net_horaire_moyen_F_plus_de_50_ans,Salaire_net_horaire_moyen_H_18_à_25_ans,Salaire_net_horaire_moyen_H_26_à_50_ans,Salaire_net_horaire_moyen_H_plus_de_50_ans,temps_acces_centralites,Complexe sportif,Piscine,"Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques",Élevage de vaches laitières,Élevage de porcins,Élevage de volailles,"Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses",Culture du riz,"Culture de légumes, de melons, de racines et de tubercules",Culture de la canne à sucre,Culture du tabac,Culture de plantes à fibres,Autres cultures non permanentes,Culture de la vigne,Culture de fruits tropicaux et subtropicaux,Culture d'agrumes,Culture de fruits à pépins et à noyau,Culture d'autres fruits d'arbres ou d'arbustes et de fruits à coque,Culture de fruits oléagineux,Culture de plantes à boissons,Autres cultures permanentes,Exploitation forestière,Extraction de pétrole brut,Extraction de gaz naturel,Extraction de minerais de fer,Extraction de minerais d'uranium et de thorium,Extraction d'autres minerais de métaux non ferreux,Transformation et conservation de la viande de boucherie,Fabrication de pesticides et d’autres produits agrochimiques,Fabrication de produits pharmaceutiques de base,Collecte des déchets non dangereux,Collecte des déchets dangereux,Traitement et élimination des déchets non dangereux,Traitement et élimination des déchets dangereux,Dépollution et autres services de gestion des déchets,"Captage, traitement et distribution d'eau",Collecte et traitement des eaux usées,Fabrication de parfums et de produits pour la toilette,Fabrication de ciment,Sidérurgie,Métallurgie des autres métaux non ferreux,Élaboration et transformation de matières nucléaires,Production d'électricité,Transport d'électricité,Distribution d'électricité,Transports routiers de fret de proximité,Restauration traditionnelle,Activités des parcs d'attractions et parcs à thèmes,Administration publique générale,nb_de_places_hebergements_tourisme,nb_dhebergements_tourisme,nb_emplois,age_0_10,age_11_17,age_18_39,age_40_79,age_80_plus,altitude_moyenne,densite_population,effectif_salaries,impots_moyen
0,01001,L'Abergement-Clémenciat,1,3,0,3,0,4,0,4,0,0,4,0,4,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [139]:
factors.to_csv("tables/factors.csv", index=False)

#factors.to_csv('factors.csv', index=False)


In [140]:
codes_geo_clean = codes_geo_2024.sort_values(by='CAN', na_position='last').drop_duplicates(subset='COM', keep='first')

# Fusionner les deux fichiers sur COM (dans codes_geo_clean) et codgeo (dans factors_qual)
facteurs_avec_canton = codes_geo_clean[['COM', 'CAN']].merge(
    factors, left_on='COM', right_on='codgeo', how='right')

facteurs_avec_canton.drop(columns=['COM'], inplace=True)
cols = ['CAN'] + [col for col in facteurs_avec_canton.columns if col != 'CAN']
facteurs_avec_canton = facteurs_avec_canton[cols]

In [141]:
# supprimer les villes d'outre mer sans canton
cantons_nan = facteurs_avec_canton[facteurs_avec_canton['CAN'].isna()]
print("Villes à supprimer : \n\n",cantons_nan)

# Liste des codgeo à exclure (DOM)
codgeo_outre_mer = [
    "97209", "97213", "97228",  # Martinique
    "97302", "97304", "97307", "97309", "97311"  # Guyane
]

# Filtrer 
facteurs_avec_canton = facteurs_avec_canton[~facteurs_avec_canton['codgeo'].isin(codgeo_outre_mer)]




Villes à supprimer : 

        CAN codgeo          libgeo  acces_medecin_generaliste  \
35     NaN  01039               0                        0.0   
467    NaN  02077               0                        0.0   
2621   NaN  08294               0                        0.0   
3062   NaN  09255               0                        0.0   
4120   NaN  12123               0                        0.0   
...    ...    ...             ...                        ...   
34993  NaN  97357     Grand-Santi                        0.0   
34994  NaN  97358      Saint-Élie                        0.0   
34995  NaN  97360          Apatou                        0.0   
34996  NaN  97361  Awala-Yalimapo                        0.3   
34997  NaN  97362      Papaichton                        0.0   

       acces_soins_infirmiers  nb_seveso  Nb_habitants  \
35                        0.0        0.0           0.0   
467                       0.0        0.0           0.0   
2621                      0.0    

In [142]:
facteurs_avec_canton.rename(columns={'CAN': 'canton'}, inplace=True)
facteurs_avec_canton['canton'] = facteurs_avec_canton['canton'].astype('string')
facteurs_avec_canton.head()

,canton,codgeo,libgeo,acces_medecin_generaliste,acces_soins_infirmiers,nb_seveso,Nb_habitants,nb_creation_entreprises_industrie_23,nb_creation_entreprises_construction_23,nb_creation_entreprises_transport_hebergement_restauration_23,nb_creation_entreprises_information_communication_23,nb_creation_entreprises_finance_asssurance_23,nb_creation_entreprises_immobilier,nb_creation_entreprises_scientifique_technique_admininistratif_23,nb_creation_entreprises_public_sante_enseignement_23,nb_creation_entreprises_autre_23,med_du_niveau_de_vie,nb_menages_fiscaux,taux_pauvrete,taux_pauvrete_proprietaire,taux_pauvrete_locataire,moy_salaire_net_horaire,moy_salaire_net_horaire_cadres_sup,moy_salaire_net_horaire_professions_intermediaires,moy_salaire_net_horaire_employes,moy_salaire_femme_net_horaire,moy_salaire_homme_net_horaire,ips_lycees,nb_ctr_tri,nb_exploitations_agri_bio,nb_france_travail,card,chirdent,derm,gastro,gynec,inf,masskine,medg,opht,orl,pedia,pneumo,psychi,psycho,radio,sgf,part_pop_eloigne_soin,Salaire_net_horaire_moyen,Salaire_net_hor_Moy_Cadres_sup,Salaire_net_hor_moy_prof_inter,Salaire_net_hor_moy_employés,Salaire_net_hor_moy_ouvriers,Salaire_net_horaire_moyen_F,Salaire_net_hor_moy_F_cadres_sup,Salaire_net_hor_moy_F_prof_inter,Salaire_net_hor_moy_F_employés,Salaire_net_hor_moy_F_ouvriers,Salaire_net_horaire_moyen_H,Salaire_net_hor_moy_H_cadres_sup,Salaire_net_hor_moy_H_prof_inter,Salaire_net_hor_moy_H_employés,Salaire_net_hor_moy_H_ouvriers,Salaire_net_horaire_moyen_18_à_25_ans,Salaire_net_horaire_moyen_26_à_50_ans,Salaire_net_horaire_moyen_plus_de_50_ans,Salaire_net_horaire_moyen_F_18_à_25_ans,Salaire_net_horaire_moyen_F_26_à_50_ans,Salaire_net_horaire_moyen_F_plus_de_50_ans,Salaire_net_horaire_moyen_H_18_à_25_ans,Salaire_net_horaire_moyen_H_26_à_50_ans,Salaire_net_horaire_moyen_H_plus_de_50_ans,temps_acces_centralites,Complexe sportif,Piscine,"Culture de plantes à épices, aromatiques, médicinales et pharmaceutiques",Élevage de vaches laitières,Élevage de porcins,Élevage de volailles,"Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses",Culture du riz,"Culture de légumes, de melons, de racines et de tubercules",Culture de la canne à sucre,Culture du tabac,Culture de plantes à fibres,Autres cultures non permanentes,Culture de la vigne,Culture de fruits tropicaux et subtropicaux,Culture d'agrumes,Culture de fruits à pépins et à noyau,Culture d'autres fruits d'arbres ou d'arbustes et de fruits à coque,Culture de fruits oléagineux,Culture de plantes à boissons,Autres cultures permanentes,Exploitation forestière,Extraction de pétrole brut,Extraction de gaz naturel,Extraction de minerais de fer,Extraction de minerais d'uranium et de thorium,Extraction d'autres minerais de métaux non ferreux,Transformation et conservation de la viande de boucherie,Fabrication de pesticides et d’autres produits agrochimiques,Fabrication de produits pharmaceutiques de base,Collecte des déchets non dangereux,Collecte des déchets dangereux,Traitement et élimination des déchets non dangereux,Traitement et élimination des déchets dangereux,Dépollution et autres services de gestion des déchets,"Captage, traitement et distribution d'eau",Collecte et traitement des eaux usées,Fabrication de parfums et de produits pour la toilette,Fabrication de ciment,Sidérurgie,Métallurgie des autres métaux non ferreux,Élaboration et transformation de matières nucléaires,Production d'électricité,Transport d'électricité,Distribution d'électricité,Transports routiers de fret de proximité,Restauration traditionnelle,Activités des parcs d'attractions et parcs à thèmes,Administration publique générale,nb_de_places_hebergements_tourisme,nb_dhebergements_tourisme,nb_emplois,age_0_10,age_11_17,age_18_39,age_40_79,age_80_plus,altitude_moyenne,densite_population,effectif_salaries,impots_moyen
0,0108,01001,L'Abergement-Clémenciat,1.8,121.5,0.0,832.0,0.0,2.0,0.0,1.0,0.0,0.0,7.0,0.0,3.0,25820.0,346.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0

In [143]:
len(facteurs_avec_canton)

35031

In [144]:
facteurs_avec_canton.to_csv("tables/factors_for_visualization.csv", index=False)